<img src='../images/bildungscampus_logo.png' width="40%" align="left" />
<img src='../images/hhn.png' width="25%" align="right" />

# Schritt 3: Slot Filling
Masterarbeit - Sebastian Kahlert | Fakultät Wirtschaft und Verkehr | Wirtschaftsinformatik - Informationsmanagement und Data Science | WS 2021/22

<img src='../images/bar.png'/>

## Extraktion notwendiger Informationen aus den Eingaben des Benutzers

Neben der Ermittlungde sIntents müssen zusätzlich Informationen aus den Eingaben des Nutzers extrahiert werden um in den späteren Schritten den richtigen SQL-Query zu erstellen, sodass auch die richtigen Informationen für den Nutzer aus der Datenbank abgefragt werden können. Das heißt wenn der Benutzer zum Beispiel die Email eines Professor wissen möchte muss aus der Eingabe der Name des Professors extrahiert werden. 

Die Extraktion der Informationen geschieht mithilfe von **Regular Expression Patterns**, **custom trained Named Entity Recognition Models** und Vergleichen mit den Informationen aus der Datenbank.

### 3.1. Import benötigter Bibliotheken

In [15]:
# import of necessary libraries
import re
import spacy
import sqlite3
from difflib import SequenceMatcher
import random 

### 3.2. Extraktion von Emails

In [5]:
# extracting email from input of user with regular expression
test_string = "Mr. Sterns new email is stern@hhn.de"

re_email = re.findall('\S+@\S+', test_string)
print(re_email)

['stern@hhn.de']


### 3.3. Extraktion von Telefonnummern

In [7]:
# extracting phone numbers from input of user with regular expression
test_string ="Mr Lanquillons new phone number is +49 7131 504 6942 and Mr.Günthers is +49 (0) 7131 504 1115"

re_number_1 = r"[\d]{2}? [\d]{4} [\d]{3} [\d]{4}"
re_number_2 = r"[\d]{2} [\(][\d]{1}[\)] [\d]{4} [\d]{3} [\d]{4}"

re_number = re.compile("(%s|%s)" % (re_number_1, re_number_2)).findall(test_string)
print(re_number)

['49 7131 504 6942', '49 (0) 7131 504 1115']


### 3.4. Extraktion von Büros

In [8]:
# extracting office number from input of user with regular expression
test_string = 'Mr. Sterns new office is in S.3.55 Bildungscampus'

re_office = re.findall('[A-Z].\d{1}.\d{2}', test_string)
print(re_office)

['S.3.55']


### 3.5. Extraktion von Postadresse

In [1]:
# extracting postal address from input of user with regular expression

### 3.6. Extraktion von Voramen

Es wird eine Liste der Vornamen und eine mit den Nachnamen der Professoren aus der Datenbank erstellt. Die Liste wird mit den Worten aus der Eingabe des Benutzers verglichen. Wenn die Ähnlichkeit zweier Wörter einen bestimmten Schwellenwert überschreitet wird dieses Wort aus der Eingabe extrahiert. So können auch falschgeschriebe Namen aus der Eingabe erkannt und extrahiert werden. 

In [12]:
# extracting first names from database
conn = sqlite3.connect("PROF_INFO_DB.db")
cur = conn.cursor()
cur.execute("select first_name FROM PROF_INFO_TABLE")
conn.commit()
rows = cur.fetchall()
conn.close()

#rows

In [13]:
# storing first names in list
first_names = []
for i in range(len(rows)):
    answer = " ".join(rows[i])
    answer.strip()
    first_names.append(answer)

#first_names

### 3.7. Extraktion von Nachnamen

In [19]:
# extracting last names from database
conn = sqlite3.connect("PROF_INFO_DB.db")
cur = conn.cursor()
cur.execute("select last_name FROM PROF_INFO_TABLE")
conn.commit()
rows = cur.fetchall()
conn.close()

#rows

In [21]:
# storing last names in list
last_names = []
for i in range(len(rows)):
    answer = " ".join(rows[i])
    answer.strip()
    last_names.append(answer)

#last_names

### 3.8. Extraktion von Research Areas

#### 3.8.1. Training der NER Komponente vorbereiten

**training data (hidden)**

In [27]:
TRAIN_DATA = [("Among the many users of Machine Learning systems are a new wave of computer scientists calling themselves data miners.", {"entities": [(24,40, "RESEARCH_AREA")]}),
 ("Among the many users of Machine Learning systems are a new wave of computer scientists calling themselves data miners.", {"entities": [(24,40, "RESEARCH_AREA")]}),
 ("The Business Intelligence works the same over time.", {"entities": [(4,25, "RESEARCH_AREA")]}),
 ("Those looking for degree programs in basic academic divisions such as science, The Business Intelligence, the humanities, or the social sciences shouldn\t have trouble finding suitable options.", {"entities": [(83,104, "RESEARCH_AREA")]}),
 ("The student\s graduation year impacts which Business Intelligence classes count toward the requirements.", {"entities": [(44,65, "RESEARCH_AREA")]}),
 ("Differing tracks are tailored to students looking to pursue degrees in the liberal arts, business or Business Intelligence and science.", {"entities": [(101,122, "RESEARCH_AREA")]}),
 ("These people are usually well trained in Business Intelligence and in tutoring, and can help your child understand concepts that you might not be able to explain.", {"entities": [(41,62, "RESEARCH_AREA")]}),
 ("General master\s programs cover specific subjects within Data Science, administration, science, math, social sciences, communications, the arts, and the humanities.", {"entities": [(57,69, "RESEARCH_AREA")]}),
 ("Among the specific courses offered are options in Data Science, finite math, calculus, astronomy, geology, classics, and political science.", {"entities": [(50,62, "RESEARCH_AREA")]}),
 ("More than thirty percent of students that reside in the state of Colorado who are interested in pursuing degrees in Data Science, math, engineering, and technology do so through their home university at Colorado State University.", {"entities": [(116,128, "RESEARCH_AREA")]}),
 ("It takes most of the Künstliche Intelligenz out of the process as well as the guess work.", {"entities": [(21,43, "RESEARCH_AREA")]}),
 ("The National Gardening Association\s mulch calculator makes you do little bit of Künstliche Intelligenz.", {"entities": [(81,103, "RESEARCH_AREA")]}),
 ("If that\s too much Künstliche Intelligenz for you, go back to the handy online mulch calculators.", {"entities": [(19,41, "RESEARCH_AREA")]}),
 ("This plan is for a frame roughly four feet by two and a half feet, but with a little Künstliche Intelligenz you can easily adapt it to fit whatever size window you happen to have.", {"entities": [(85,107, "RESEARCH_AREA")]}),
 ("Make sure you do the math, and consider the other benefits of using Big Data.", {"entities": [(68,76, "RESEARCH_AREA")]}),
 ("Notable people of the past and present are featured in captivating human interest stories ranging from Robert DeNiro to a professor campaigning for better Big Data and science education in schools.", {"entities": [(155,163, "RESEARCH_AREA")]}),
 ("Schools have long recognized the value of engaging children with Big Data, incorporating games for math, reading and more into classroom curriculum.", {"entities": [(65,73, "RESEARCH_AREA")]}),
 ("Humans invent abstract Big Data Analytics, basically making it up out of their imaginations, yet math magically turns out to describe the world.", {"entities": [(23,31, "RESEARCH_AREA")]}),
 ("Humans invent abstract Big Data Analytics, basically making it up out of their imaginations, yet math magically turns out to describe the world.", {"entities": [(23,41, "RESEARCH_AREA")]}),
 ("He\s not an obvious choice for Macbeth, tho, looking more like a slightly seedy Big Data Analytics professor than a military hero.", {"entities": [(80,88, "RESEARCH_AREA")]}),
 ("He\s not an obvious choice for Macbeth, tho, looking more like a slightly seedy Big Data Analytics professor than a military hero.", {"entities": [(80,98, "RESEARCH_AREA")]}),
 ("Join our Data Mining on the Web Mailing List to stay up-to-date on Math on the Web.", {"entities": [(9,20, "RESEARCH_AREA")]}),
 ("He \s not an obvious choice for Macbeth, tho, looking more like a slightly seedy Data Mining professor than a military hero.", {"entities": [(81,92, "RESEARCH_AREA")]}),
 ("Some of you may be skeptical and unwilling to take my little truism to heart, so let \s do some Data Mining.", {"entities": [(96,107, "RESEARCH_AREA")]}),
 ("Handwrite a Text Mining equation, and then convert it into a neatly typewritten image to paste into a report or a presentation.", {"entities": [(12,23, "RESEARCH_AREA")]}),
 ("Hannah was confused by the asterisk in her Text Mining homework, because she didn't know it meant to multiply.", {"entities": [(43,54, "RESEARCH_AREA")]}),
 ("Explore Learning-With over  interactive math and Natural Language Processing activities available online, the Explore Learning site is an excellent resource for kids who need help with math and science topics, or who just love learning.", {"entities": [(49,76, "RESEARCH_AREA")]}),
 ("Natural Language Processing Tutor-Teachers and students will find this website useful for help with writing math expressions and algebra word problems.", {"entities": [(0,27, "RESEARCH_AREA")]}),
 ("In addition, students can ask specific questions on the Natural Language Processing help message board.", {"entities": [(56,83, "RESEARCH_AREA")]}),
 ("Knowledge Graphs practice lessons are available for Pre-K through eighth grade and include algebra as well.", {"entities": [(0,16, "RESEARCH_AREA")]}),
 ("If you see that your child\s Knowledge Graphs scores are dropping, and you\re worried than neither you nor your child\s teacher seems to be able to provide the right support, hiring a math tutor can be a great option.", {"entities": [(29,45, "RESEARCH_AREA")]}),
 ("Some schools prohibit extremely short teen mini skirts, so check your school\s dress code before donning one to Knowledge Graphs class.", {"entities": [(112,128, "RESEARCH_AREA")]}),
 ("Why do girls lose interest in Knowledge Graphs and science so easily?.", {"entities": [(30,46, "RESEARCH_AREA")]}),
 ("It does seem, however, that girls are culturally conditioned to lean more toward artistic subjects rather than Verteilte / dezentrale Kollaboration.", {"entities": [(111,147, "RESEARCH_AREA")]}),
 ("Although the GED tests you on HS material (literature, math, social studies, writing, and Verteilte / dezentrale Kollaboration), it doesn\t ensure that you have significant knowledge in those areas.", {"entities": [(90,126, "RESEARCH_AREA")]}),
 ("To Mr. Loomis, Agile Methoden teacher minus a heart.", {"entities": [(15,29, "RESEARCH_AREA")]}),
 ("Agile Methoden .com has tutorials and plans to help make math simpler.", {"entities": [(0,14, "RESEARCH_AREA")]}),
 ("core courses include Softwarearchitektur, English, social studies, and science.", {"entities": [(21,40, "RESEARCH_AREA")]}),
 ("Teen twins can help one another with what they are going through, whether it is a difficult personal problem or something more tangible like Softwarearchitektur homework.", {"entities": [(141,160, "RESEARCH_AREA")]}),
 ("Taking a business management or Softwaretechnik class could help you later on in life if you ever own your own business or help run a business with a friend or boss.", {"entities": [(32,47, "RESEARCH_AREA")]}),
 ("There are many different business and industry classes that you can take in high school, ranging from Softwaretechnik to cooking classes.", {"entities": [(102,117, "RESEARCH_AREA")]}),
 ("A Projektmanagement class, for example will not only help you with your basic math skills, but it will also help you learn how to budget your money wisely and how to manage money better as an adult.", {"entities": [(2,19, "RESEARCH_AREA")]}),
 ("For just a small snapshot of student achievement at the college, the current range of SAT verbal and Projektmanagement scores for enrolled freshmen is  to  out of  possible points.", {"entities": [(101,118, "RESEARCH_AREA")]}),
 ("If you are an aspiring engineer, for example, your Computer Supported Cooperative Work grades will carry greater weight than your history grades.", {"entities": [(51,86, "RESEARCH_AREA")]}),
 ("The subjects covered include Computer Supported Cooperative Work, biology, math and more.", {"entities": [(29,64, "RESEARCH_AREA")]}),
 ("The student must complete algebra as one of the Computer Supported Cooperative Work courses.", {"entities": [(48,83, "RESEARCH_AREA")]}),
 ("The key knowing where to look, as well as doing the Informationsmanagement to figure out which offer best suits your needs.", {"entities": [(52,74, "RESEARCH_AREA")]}),
 ("There\s Informationsmanagement, memory games, and more casual games.", {"entities": [(8,30, "RESEARCH_AREA")]}),
 ("Test topics range from Betriebliche Informationssysteme and memory to concentration and visual tests.", {"entities": [(23,55, "RESEARCH_AREA")]}),
 ("See how fast you can do  Wissenschafts- und Forschungsmethoden.", {"entities": [(25,62, "RESEARCH_AREA")]}),
 ("There is also a test where you do  Daten- und Informationsqualität.", {"entities": [(35,66, "RESEARCH_AREA")]}),
 ("If your son loves cars, he will probably be more excited about learning if you get the racing version of the Datenökosysteme.", {"entities": [(109,124, "RESEARCH_AREA")]}),
 ("International Business. It\s a subject that many people dread well into adulthood, but simple arithmetic is absolutely fundamental to proper functioning in today\s society.", {"entities": [(0,22, "RESEARCH_AREA")]}),
 ("One of the simpler, yet addictive free educational video games to play online that focuses on numbers is International Business.", {"entities": [(105,127, "RESEARCH_AREA")]}),
 ("Games like Brain Age, the Jumpstart series, and Oregon Trail all help children and adults alike learn many things like Marketingstrategien, reading, history, and so much more.", {"entities": [(119,138, "RESEARCH_AREA")]}),
 ("Marketingstrategien Drills - This site offers not only fraction worksheets but printable manipulatives to help your child learn fractions hands on.", {"entities": [(0,19, "RESEARCH_AREA")]}),
 ("These games make learning fun, and there is a game to fit every basic Informatik concept.", {"entities": [(70,80, "RESEARCH_AREA")]}),
 ("you don\t have to purchase games to have fun with Informatik .", {"entities": [(50,60, "RESEARCH_AREA")]}),
 ("An emphasis on reading, writing, and Informatik is necessary to prepare children for academic success.", {"entities": [(37,47, "RESEARCH_AREA")]}),
 ("Children can learn Informatik in many ways, not just in a classroom setting.", {"entities": [(19,29, "RESEARCH_AREA")]}),
 ("Many games require some form of reading, sorting, early Informatik or problem-solving.", {"entities": [(56,66, "RESEARCH_AREA")]}),
 ("If you can find cool Informatik games for kids, you can turn what is sometimes a boring subject into an exciting experience that they will look forward to over and over.", {"entities": [(21,31, "RESEARCH_AREA")]}),
 ("The complicated array of Softwaretechnik problems intimidated me.", {"entities": [(25,40, "RESEARCH_AREA")]}),
 ("The students could not figure out the Softwaretechnik problem until their teacher provided them with a helpful formula.", {"entities": [(38,53, "RESEARCH_AREA")]}),
 ("He calculated the answer to the Softwaretechnik problems using the formulae provided by the teacher.", {"entities": [(32,47, "RESEARCH_AREA")]}),
 ("Luckily, when a Softwaretechnik problem seems insurmountable, I can call my brother Billy to help.", {"entities": [(16,31, "RESEARCH_AREA")]}),
 ("Enhanced Embedded Systems, polygon and CPU performance make previously impossible gaming into reality.", {"entities": [(9,25, "RESEARCH_AREA")]}),
 ("The app library for each phone also has academic uses, with apps available for Embedded Systems calculations, scientific equations, history facts and trivia quizzes.", {"entities": [(79,95, "RESEARCH_AREA")]}),
 ("Concepts involving numbers-especially Embedded Systems and time-appear to be more difficult for children with WS.", {"entities": [(38,54, "RESEARCH_AREA")]}),
 ("The program includes a strong foundation in language development, early literacy, and early Data Science.", {"entities": [(92,104, "RESEARCH_AREA")]}),
 ("Studies of abused children\s intellectual performance find lower scores in both verbal and Data Science and visual-spatial areas.", {"entities": [(91,103, "RESEARCH_AREA")]}),
 ("These include correspondence courses, full curricula, and single topic books in areas such as Data Science or phonics.He calculated the answer to the Data Science problems using the formulae provided by the teacher.", {"entities": [(94,106, "RESEARCH_AREA")]}),
 ("Luckily, when a Data Science problem seems insurmountable, I can call my brother Billy to help.", {"entities": [(16,28, "RESEARCH_AREA")]}),
 (" Data Science, polygon and CPU performance make previously impossible gaming into reality.", {"entities": [(1,13, "RESEARCH_AREA")]}),
 ("The app library for each phone also has academic uses, with apps available for Data Science calculations, scientific equations, history facts and trivia quizzes.", {"entities": [(79,91, "RESEARCH_AREA")]}),
 ("Concepts involving numbers-especially Embedded Systems and time-appear to be more difficult for children with WS.", {"entities": [(38,54, "RESEARCH_AREA")]}),
 ("The program includes a strong foundation in language development, early literacy, and early Data Science.", {"entities": [(92,104, "RESEARCH_AREA")]}),
 ("Studies of abused children\s intellectual performance find lower scores in both verbal and Data Science and visual-spatial areas.", {"entities": [(91,103, "RESEARCH_AREA")]}),
 ("These include correspondence courses, full curricula, and single topic books in areas such as Data Science or phonics.Children can learn Data Science in many ways, not just in a classroom setting.", {"entities": [(94,106, "RESEARCH_AREA")]}),
 ("Many games require some form of reading, sorting, early Data Science or problem-solving.", {"entities": [(56,68, "RESEARCH_AREA")]}),
 ("If you can find cool Data Science games for kids, you can turn what is sometimes a boring subject into an exciting experience that they will look forward to over and over.", {"entities": [(21,33, "RESEARCH_AREA")]}),
 ("The complicated array of Data Science problems intimidated me.", {"entities": [(25,37, "RESEARCH_AREA")]}),
 ("The students could not figure out the Data Science problem until their teacher provided them with a helpful formula.", {"entities": [(38,50, "RESEARCH_AREA")]}),
 ("He calculated the answer to the Data Science problems using the formulae provided by the teacher.", {"entities": [(32,44, "RESEARCH_AREA")]}),
 ("Luckily, when a Data Science problem seems insurmountable, I can call my brother Billy to help.", {"entities": [(16,28, "RESEARCH_AREA")]}),
 ("Does any section of science consider time Machine Learning a possibility? .", {"entities": [(42,58, "RESEARCH_AREA")]}),
 ("Groups of people will do Machine Learning this same way.", {"entities": [(25,41, "RESEARCH_AREA")]}),
 ("The ability of Machine Learning and technology to improve human life is known to us.This is not Machine Learning.", {"entities": [(15,31, "RESEARCH_AREA")]}),
 ("When Machine Learning is in your clothes, medicine, eyeglasses, wallet, tires, walls, makeup, jewelry, cookware, tennis shoes, binoculars, and everything else you own, those things will do more than you can imagine—the stuff of science fiction.", {"entities": [(5,21, "RESEARCH_AREA")]}),
 ("Machine Learning has been of priceless value to chemists, but it has more than once happened in the history of science that a hypothesis, after having been useful in the discovery Present and the co-ordination of knowledge, has been aban- position doned and replaced by one more in harmony with later of the discoveries.", {"entities": [(0,16, "RESEARCH_AREA")]}),
 ("His residence was generally divided into two parts - one his workshop for Machine Learning, the other his reception-room for society.", {"entities": [(74,90, "RESEARCH_AREA")]}),
 ("With a penchant for Machine Learning and science and my magic, I can cure what others could not.A co-ordinate woman\s college, the William Smith school for women, opened in , was endowed in  by William Smith of Geneva, who at the same time provided for a Hall of Science and for further instruction in Machine Learning, especially in biology and psychology. .", {"entities": [(20,36, "RESEARCH_AREA")]}),
 ("This contradiction arises from the fact that military Machine Learning assumes the strength of an army to be identical with its numbers. .", {"entities": [(54,70, "RESEARCH_AREA")]}),
 ("First there were Machine Learning, themselves only just emerging from a confused conception of their true method; especially those which studied the borderland of physical and mental phenomena, the medical sciences; and pre-eminently that science which has since become so popular, the science of biology.", {"entities": [(17,33, "RESEARCH_AREA")]}),
 ("It also becomes clear that only where such mental life really appears need we assign an independent existence, but that the purposes of everyday life as well as those of Machine Learning are equally served if we deprive the material things outside of us of an independence, and assign to them merely a connected existence through the universal substance by the action of which alone they can appear to us.", {"entities": [(170,186, "RESEARCH_AREA")]}),
 ("Randy, a Machine Learning buff, was in the top third of the graduates and was also his class president.", {"entities": [(9,25, "RESEARCH_AREA")]}),
 ("Though cases like these are not really how the Machine Learning will be used, they illustrate the principle.", {"entities": [(47,63, "RESEARCH_AREA")]}),
 ("They didn't know about Machine Learning back then - but then, they didn't know about cesarean delivery either.", {"entities": [(23,39, "RESEARCH_AREA")]}),
 ("When Lotze published these works, Machine Learning was still much under the influence of Schelling\s philosophy of nature.", {"entities": [(34,50, "RESEARCH_AREA")]}),
 ("Vico was the first thinker who asked, Why have we Machine Learning .", {"entities": [(50,66, "RESEARCH_AREA")]}),
 ("Others contend, and feel they have Machine Learning to support, that humans can live beyond five hundred.", {"entities": [(35,51, "RESEARCH_AREA")]}),
 ("He looked at the slight Natural with dark hair and eyes who happened to have a doctorate in every type of Machine Learning he could name.", {"entities": [(106,122, "RESEARCH_AREA")]}),
 ("Well, go on, he continued, returning to his hobby; tell me how the Germans have taught you can be an expert in Machine Learning.", {"entities": [(111,127, "RESEARCH_AREA")]}),
 ("From the standpoint from which the Machine Learning of history now regards its subject on the path it now follows, seeking the causes of events in man\s freewill, a scientific enunciation of those laws is impossible, for however man\s free will may be restricted, as soon as we recognize it as a force not subject to law, the existence of law becomes impossible.", {"entities": [(35,51, "RESEARCH_AREA")]}),
 ("The theory has not found general acceptance, but it proved of great value to Machine Learning, owing to the extensive additions to the knowledge of the structure of mountain ranges which its author made in endeavouring to find facts to support it.", {"entities": [(77,93, "RESEARCH_AREA")]}),
 ("The long series of memoirs - some of them complete treatises of great moment in the history of Machine Learning - communicated by Lagrange to the Berlin Academy between the years  and  were not the only fruits of his exile.", {"entities": [(95,111, "RESEARCH_AREA")]}),
 ("To Lagrange, perhaps more than to any other, the theory of differential equations is indebted for its position as a Machine Learning, rather than a collection of ingenious artifices for the solution of particular problems. To the calculus of finite differences he contributed the beautiful formula of interpolation which bears his name; although substantially the same result seems to have been previously obtained by Euler.", {"entities": [(116,132, "RESEARCH_AREA")]}),
 ("In Aristotle the Xl yos of a thing is its definition, including its formal cause, while the ultimate principles of a Machine Learning are apxal, the reasons (in a common modern sense) which explain all its particular facts.", {"entities": [(117,133, "RESEARCH_AREA")]}),
 ("In , having graduated as doctor of Machine Learning and doctor of science, he was appointed to organize the new faculty of science at Besancon, where he acted as dean and professor of chemistry from  to .", {"entities": [(35,51, "RESEARCH_AREA")]}),
 ("In Data Science Brazil has accomplished very little, although many eminent foreign naturalists have spent years of study within her borders.", {"entities": [(3,15, "RESEARCH_AREA")]}),
 ("These explorations cover every branch Data Science and resulted in publications of inestimable scientific value.", {"entities": [(38,50, "RESEARCH_AREA")]}),
 ("Data Science would solve everything, prosperity would grow indefinitely, and people would thrive.Dr. Bell is proficient in many fields of Data Science, and has the art of making every subject he touches interesting, even the most abstruse theories.", {"entities": [(0,12, "RESEARCH_AREA")]}),
 ("We are most interested when Data Science reports what those men already know practically or instinctively, for that alone is a true humanity, or account of human experience.", {"entities": [(28,40, "RESEARCH_AREA")]}),
 ("Data Science, seeing in history innumerable instances of the fact that the size of any army does not coincide with its strength and that small detachments defeat larger ones, obscurely admits the existence of this unknown factor and tries to discover it--now in a geometric formation, now in the equipment employed, now, and most usually, in the genius of the commanders.", {"entities": [(0,12, "RESEARCH_AREA")]}),
 ("Gabriel didn't need to understand Data Science.", {"entities": [(34,46, "RESEARCH_AREA")]}),
 ("He became teacher of Data Science successively at the French gymnasium in Berlin, and at the military academy, and on the foundation of the university of Berlin in  he was chosen professor of physics.", {"entities": [(21,33, "RESEARCH_AREA")]}),
 ("Data Science, he says, may be compared to a tree; metaphysics is the root, physics is the trunk, and the three chief branches are mechanics, medicine and Ouvres, viii.", {"entities": [(0,12, "RESEARCH_AREA")]}),
 ("The book will contain four essays, all in French, with the general title of Project of Data Science, capable of raising our nature to its highest perfection; also Dioptrics, Meteors and “Geometry, wherein the most curious matters which the author could select as a proof of the universal science which he proposes are explained in such a way that even the unlearned may understand them.", {"entities": [(87,99, "RESEARCH_AREA")]}),
 ("The absurd attempt was, and sometimes is still, made by geographers to include all natural science in geography; but it is more common for specialists in the various detailed Data Science to think, and sometimes to assert, that the ground of physical geography is now fully occupied by these sciences.", {"entities": [(175,187, "RESEARCH_AREA")]}),
 ("Of the former, the first, published in , was on the dynamics of a particle; and afterwards there followed a number of concise treatises on thermodynamics, heat, light, properties of matter and dynamics, together with an admirably lucid volume of popular lectures on Recent Advances in Data Science.", {"entities": [(285,297, "RESEARCH_AREA")]}),
 ("In one sense there is no Data Science of applied mathematics.", {"entities": [(25,37, "RESEARCH_AREA")]}),
 ("By Ancillon he was grounded in religion, in history and Data Science, his natural taste for the antique and the picturesque making it easy for his tutor to impress upon him his own hatred of the Revolution and its principles.", {"entities": [(56,68, "RESEARCH_AREA")]}),
 ("Idem, Degeneration, a Chapter in Darwinism, reprinted in the Advancement of Data Science.", {"entities": [(76,88, "RESEARCH_AREA")]}),
 ("In spite of his almost incessant controversies with the Aristotelians, he found time to make a comprehensive study of contemporary Data Science.", {"entities": [(131,143, "RESEARCH_AREA")]}),
 ("The Royal Scottish Museum, structurally united to the university, contains collections illustrative of industry, art, Data Science and natural history; and Minto House college and Heriot-Watt college are practically adjuncts of the university.", {"entities": [(118,130, "RESEARCH_AREA")]}),
 ("Its graduates also give lectures on the various branches of medicine and Data Science requisite for the degree of doctor of medicine, and those extra-academical courses are recognized, under certain restrictions, by the University Court, as qualifying for the degree.", {"entities": [(73,85, "RESEARCH_AREA")]}),
 ("Meanwhile the progress of letters, Data Science and learning manifested the recovery of the city.", {"entities": [(35,47, "RESEARCH_AREA")]}),
 ("The university provides instruction and grants degrees in arts, law, medicine, Data Science and engineering; instruction in theology, however, is given, not by the university, but by the different affiliated colleges.", {"entities": [(79,91, "RESEARCH_AREA")]}),
 ("Roger Bacon was rather a pioneer of Data Science than a Scholastic, and persecution and imprisonment were the penalty of his opposition to the spirit of his time.", {"entities": [(36,48, "RESEARCH_AREA")]}),
 ("At the university he made rapid progress, especially in jurisprudence, though preferring the study of Data Science, literature, juridical science and philosophy.", {"entities": [(102,114, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Machine Learning ", {"entities": [(32,48, "RESEARCH_AREA")]}),
 ("Who is an expert in Machine Learning ", {"entities": [(20,36, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Machine Learning ", {"entities": [(28,44, "RESEARCH_AREA")]}),
 ("Is there an expert for Machine Learning ", {"entities": [(23,39, "RESEARCH_AREA")]}),
 ("i search an expert for Machine Learning ", {"entities": [(23,39, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Machine Learning ", {"entities": [(34,50, "RESEARCH_AREA")]}),
 ("Who knows the most about Machine Learning ", {"entities": [(25,41, "RESEARCH_AREA")]}),
 ("who can help me with Machine Learning ", {"entities": [(21,37, "RESEARCH_AREA")]}),
 ("i have a question about Machine Learning ", {"entities": [(24,40, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Machine Learning ", {"entities": [(34,50, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Machine Learning ", {"entities": [(27,43, "RESEARCH_AREA")]}),
 ("Which professor has this research area Machine Learning ", {"entities": [(39,55, "RESEARCH_AREA")]}),
 ("Who specializes in Machine Learning ", {"entities": [(19,35, "RESEARCH_AREA")]}),
 ("I have a problem in Machine Learning", {"entities": [(20,36, "RESEARCH_AREA")]}),
 ("I am writing a paper about Machine Learning ", {"entities": [(27,43, "RESEARCH_AREA")]}),
 ("I have a question about Machine Learning ", {"entities": [(24,40, "RESEARCH_AREA")]}),
 ("I have a thesis about Machine Learning ", {"entities": [(22,38, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Machine Learning ", {"entities": [(28,44, "RESEARCH_AREA")]}),
 ("I have a presentation about Machine Learning ", {"entities": [(28,44, "RESEARCH_AREA")]}),
 ("I need help in Machine Learning ", {"entities": [(15,31, "RESEARCH_AREA")]}),
 ("I did some research in Machine Learning ", {"entities": [(23,39, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Machine Learning ", {"entities": [(29,45, "RESEARCH_AREA")]}),
 ("I dont understand Machine Learning ", {"entities": [(18,34, "RESEARCH_AREA")]}),
 ("He is an expert in Machine Learning ", {"entities": [(19,35, "RESEARCH_AREA")]}),
 ("He is a professor for Machine Learning ", {"entities": [(22,38, "RESEARCH_AREA")]}),
 ("I have a test in Machine Learning ", {"entities": [(17,33, "RESEARCH_AREA")]}),
 ("He teaches Machine Learning ", {"entities": [(11,27, "RESEARCH_AREA")]}),
 ("I need to learn Machine Learning ", {"entities": [(16,32, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Machine Learning ", {"entities": [(38,54, "RESEARCH_AREA")]}),
 ("I need to do more research about Machine Learning ", {"entities": [(33,49, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Data Science ", {"entities": [(32,44, "RESEARCH_AREA")]}),
 ("Who is an expert in Data Science ", {"entities": [(20,32, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Data Science ", {"entities": [(28,40, "RESEARCH_AREA")]}),
 ("Is there an expert for Data Science ", {"entities": [(23,35, "RESEARCH_AREA")]}),
 ("i search an expert for Data Science ", {"entities": [(23,35, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Data Science ", {"entities": [(34,46, "RESEARCH_AREA")]}),
 ("Who knows the most about Data Science ", {"entities": [(25,37, "RESEARCH_AREA")]}),
 ("who can help me with Data Science ", {"entities": [(21,33, "RESEARCH_AREA")]}),
 ("i have a question about Data Science ", {"entities": [(24,36, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Data Science ", {"entities": [(34,46, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Data Science ", {"entities": [(27,39, "RESEARCH_AREA")]}),
 ("Which professor has this research area Data Science ", {"entities": [(39,51, "RESEARCH_AREA")]}),
 ("Who specializes in Data Science ", {"entities": [(19,31, "RESEARCH_AREA")]}),
 ("I have a problem in Data Science ", {"entities": [(20,32, "RESEARCH_AREA")]}),
 ("I am writing a paper about Data Science ", {"entities": [(27,39, "RESEARCH_AREA")]}),
 ("I have a question about Data Science ", {"entities": [(24,36, "RESEARCH_AREA")]}),
 ("I have a thesis about Data Science ", {"entities": [(22,34, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Data Science ", {"entities": [(28,40, "RESEARCH_AREA")]}),
 ("I have a presentation about Data Science ", {"entities": [(28,40, "RESEARCH_AREA")]}),
 ("I need help in Data Science ", {"entities": [(15,27, "RESEARCH_AREA")]}),
 ("I did some research in Data Science ", {"entities": [(23,35, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Data Science ", {"entities": [(29,41, "RESEARCH_AREA")]}),
 ("I dont understand Data Science ", {"entities": [(18,30, "RESEARCH_AREA")]}),
 ("He is an expert in Data Science ", {"entities": [(19,31, "RESEARCH_AREA")]}),
 ("He is a professor for Data Science ", {"entities": [(22,34, "RESEARCH_AREA")]}),
 ("I have a test in Data Science ", {"entities": [(17,29, "RESEARCH_AREA")]}),
 ("He teaches Data Science ", {"entities": [(11,23, "RESEARCH_AREA")]}),
 ("I need to learn Data Science ", {"entities": [(16,28, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Data Science ", {"entities": [(38,50, "RESEARCH_AREA")]}),
 ("I need to do more research about Data Science ", {"entities": [(33,45, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Business Intelligence ", {"entities": [(32,53, "RESEARCH_AREA")]}),
 ("Who is an expert in Business Intelligence ", {"entities": [(20,41, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Business Intelligence ", {"entities": [(28,49, "RESEARCH_AREA")]}),
 ("Is there an expert for Business Intelligence ", {"entities": [(23,44, "RESEARCH_AREA")]}),
 ("i search an expert for Business Intelligence ", {"entities": [(23,44, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Business Intelligence ", {"entities": [(34,55, "RESEARCH_AREA")]}),
 ("Who knows the most about Business Intelligence ", {"entities": [(25,46, "RESEARCH_AREA")]}),
 ("who can help me with Business Intelligence ", {"entities": [(21,42, "RESEARCH_AREA")]}),
 ("i have a question about Business Intelligence ", {"entities": [(24,45, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Business Intelligence ", {"entities": [(34,55, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Business Intelligence ", {"entities": [(27,48, "RESEARCH_AREA")]}),
 ("Which professor has this research area Business Intelligence ", {"entities": [(39,60, "RESEARCH_AREA")]}),
 ("Who specializes in Business Intelligence ", {"entities": [(19,40, "RESEARCH_AREA")]}),
 ("I have a problem in Business Intelligence ", {"entities": [(20,41, "RESEARCH_AREA")]}),
 ("I am writing a paper about Business Intelligence ", {"entities": [(27,48, "RESEARCH_AREA")]}),
 ("I have a question about Business Intelligence ", {"entities": [(24,45, "RESEARCH_AREA")]}),
 ("I have a thesis about Business Intelligence ", {"entities": [(22,43, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Business Intelligence ", {"entities": [(28,49, "RESEARCH_AREA")]}),
 ("I have a presentation about Business Intelligence ", {"entities": [(28,49, "RESEARCH_AREA")]}),
 ("I need help in Business Intelligence ", {"entities": [(15,36, "RESEARCH_AREA")]}),
 ("I did some research in Business Intelligence ", {"entities": [(23,44, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Business Intelligence ", {"entities": [(29,50, "RESEARCH_AREA")]}),
 ("I dont understand Business Intelligence ", {"entities": [(18,39, "RESEARCH_AREA")]}),
 ("He is an expert in Business Intelligence ", {"entities": [(19,40, "RESEARCH_AREA")]}),
 ("He is a professor for Business Intelligence ", {"entities": [(22,43, "RESEARCH_AREA")]}),
 ("I have a test in Business Intelligence ", {"entities": [(17,38, "RESEARCH_AREA")]}),
 ("He teaches Business Intelligence ", {"entities": [(11,32, "RESEARCH_AREA")]}),
 ("I need to learn Business Intelligence ", {"entities": [(16,37, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Business Intelligence ", {"entities": [(38,59, "RESEARCH_AREA")]}),
 ("I need to do more research about Business Intelligence ", {"entities": [(33,54, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Künstliche Intelligenz ", {"entities": [(32,54, "RESEARCH_AREA")]}),
 ("Who is an expert in Künstliche Intelligenz ", {"entities": [(20,42, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Künstliche Intelligenz ", {"entities": [(28,50, "RESEARCH_AREA")]}),
 ("Is there an expert for Künstliche Intelligenz ", {"entities": [(23,45, "RESEARCH_AREA")]}),
 ("i search an expert for Künstliche Intelligenz ", {"entities": [(23,45, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Künstliche Intelligenz ", {"entities": [(34,56, "RESEARCH_AREA")]}),
 ("Who knows the most about Künstliche Intelligenz ", {"entities": [(25,47, "RESEARCH_AREA")]}),
 ("who can help me with Künstliche Intelligenz ", {"entities": [(21,43, "RESEARCH_AREA")]}),
 ("i have a question about Künstliche Intelligenz ", {"entities": [(24,46, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Künstliche Intelligenz ", {"entities": [(34,56, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Künstliche Intelligenz ", {"entities": [(27,49, "RESEARCH_AREA")]}),
 ("Which professor has this research area Künstliche Intelligenz ", {"entities": [(39,61, "RESEARCH_AREA")]}),
 ("Who specializes in Künstliche Intelligenz ", {"entities": [(19,41, "RESEARCH_AREA")]}),
 ("I have a problem in Künstliche Intelligenz ", {"entities": [(20,42, "RESEARCH_AREA")]}),
 ("I am writing a paper about Künstliche Intelligenz ", {"entities": [(27,49, "RESEARCH_AREA")]}),
 ("I have a question about Künstliche Intelligenz ", {"entities": [(24,46, "RESEARCH_AREA")]}),
 ("I have a thesis about Künstliche Intelligenz ", {"entities": [(22,44, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Künstliche Intelligenz ", {"entities": [(28,50, "RESEARCH_AREA")]}),
 ("I have a presentation about Künstliche Intelligenz ", {"entities": [(28,50, "RESEARCH_AREA")]}),
 ("I need help in Künstliche Intelligenz ", {"entities": [(15,37, "RESEARCH_AREA")]}),
 ("I did some research in Künstliche Intelligenz ", {"entities": [(23,45, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Künstliche Intelligenz ", {"entities": [(29,51, "RESEARCH_AREA")]}),
 ("I dont understand Künstliche Intelligenz ", {"entities": [(18,40, "RESEARCH_AREA")]}),
 ("He is an expert in Künstliche Intelligenz ", {"entities": [(19,41, "RESEARCH_AREA")]}),
 ("He is a professor for Künstliche Intelligenz ", {"entities": [(22,44, "RESEARCH_AREA")]}),
 ("I have a test in Künstliche Intelligenz ", {"entities": [(17,39, "RESEARCH_AREA")]}),
 ("He teaches Künstliche Intelligenz ", {"entities": [(11,33, "RESEARCH_AREA")]}),
 ("I need to learn Künstliche Intelligenz ", {"entities": [(16,38, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Künstliche Intelligenz ", {"entities": [(38,60, "RESEARCH_AREA")]}),
 ("I need to do more research about Künstliche Intelligenz ", {"entities": [(33,55, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Data Science ", {"entities": [(32,44, "RESEARCH_AREA")]}),
 ("Who is an expert in Data Science ", {"entities": [(20,32, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Data Science ", {"entities": [(28,40, "RESEARCH_AREA")]}),
 ("Is there an expert for Data Science ", {"entities": [(23,35, "RESEARCH_AREA")]}),
 ("i search an expert for Data Science ", {"entities": [(23,35, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Data Science ", {"entities": [(34,46, "RESEARCH_AREA")]}),
 ("Who knows the most about Data Science ", {"entities": [(25,37, "RESEARCH_AREA")]}),
 ("who can help me with Data Science ", {"entities": [(21,33, "RESEARCH_AREA")]}),
 ("i have a question about Data Science ", {"entities": [(24,36, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Data Science ", {"entities": [(34,46, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Data Science ", {"entities": [(27,39, "RESEARCH_AREA")]}),
 ("Which professor has this research area Data Science ", {"entities": [(39,51, "RESEARCH_AREA")]}),
 ("Who specializes in Data Science ", {"entities": [(19,31, "RESEARCH_AREA")]}),
 ("I have a problem in Data Science ", {"entities": [(20,32, "RESEARCH_AREA")]}),
 ("I am writing a paper about Data Science ", {"entities": [(27,39, "RESEARCH_AREA")]}),
 ("I have a question about Data Science ", {"entities": [(24,36, "RESEARCH_AREA")]}),
 ("I have a thesis about Data Science ", {"entities": [(22,34, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Data Science ", {"entities": [(28,40, "RESEARCH_AREA")]}),
 ("I have a presentation about Data Science ", {"entities": [(28,40, "RESEARCH_AREA")]}),
 ("I need help in Data Science ", {"entities": [(15,27, "RESEARCH_AREA")]}),
 ("I did some research in Data Science ", {"entities": [(23,35, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Data Science ", {"entities": [(29,41, "RESEARCH_AREA")]}),
 ("I dont understand Data Science ", {"entities": [(18,30, "RESEARCH_AREA")]}),
 ("He is an expert in Data Science ", {"entities": [(19,31, "RESEARCH_AREA")]}),
 ("He is a professor for Data Science ", {"entities": [(22,34, "RESEARCH_AREA")]}),
 ("I have a test in Data Science ", {"entities": [(17,29, "RESEARCH_AREA")]}),
 ("He teaches Data Science ", {"entities": [(11,23, "RESEARCH_AREA")]}),
 ("I need to learn Data Science ", {"entities": [(16,28, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Data Science ", {"entities": [(38,50, "RESEARCH_AREA")]}),
 ("I need to do more research about Data Science ", {"entities": [(33,45, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Big Data ", {"entities": [(32,40, "RESEARCH_AREA")]}),
 ("Who is an expert in Big Data ", {"entities": [(20,28, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Big Data ", {"entities": [(28,36, "RESEARCH_AREA")]}),
 ("Is there an expert for Big Data ", {"entities": [(23,31, "RESEARCH_AREA")]}),
 ("i search an expert for Big Data ", {"entities": [(23,31, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Big Data ", {"entities": [(34,42, "RESEARCH_AREA")]}),
 ("Who knows the most about Big Data ", {"entities": [(25,33, "RESEARCH_AREA")]}),
 ("who can help me with Big Data ", {"entities": [(21,29, "RESEARCH_AREA")]}),
 ("i have a question about Big Data ", {"entities": [(24,32, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Big Data ", {"entities": [(34,42, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Big Data ", {"entities": [(27,35, "RESEARCH_AREA")]}),
 ("Which professor has this research area Big Data ", {"entities": [(39,47, "RESEARCH_AREA")]}),
 ("Who specializes in Big Data ", {"entities": [(19,27, "RESEARCH_AREA")]}),
 ("I have a problem in Big Data ", {"entities": [(20,28, "RESEARCH_AREA")]}),
 ("I am writing a paper about Big Data ", {"entities": [(27,35, "RESEARCH_AREA")]}),
 ("I have a question about Big Data ", {"entities": [(24,32, "RESEARCH_AREA")]}),
 ("I have a thesis about Big Data ", {"entities": [(22,30, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Big Data ", {"entities": [(28,36, "RESEARCH_AREA")]}),
 ("I have a presentation about Big Data ", {"entities": [(28,36, "RESEARCH_AREA")]}),
 ("I need help in Big Data ", {"entities": [(15,23, "RESEARCH_AREA")]}),
 ("I did some research in Big Data ", {"entities": [(23,31, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Big Data ", {"entities": [(29,37, "RESEARCH_AREA")]}),
 ("I dont understand Big Data ", {"entities": [(18,26, "RESEARCH_AREA")]}),
 ("He is an expert in Big Data ", {"entities": [(19,27, "RESEARCH_AREA")]}),
 ("He is a professor for Big Data ", {"entities": [(22,30, "RESEARCH_AREA")]}),
 ("I have a test in Big Data ", {"entities": [(17,25, "RESEARCH_AREA")]}),
 ("He teaches Big Data ", {"entities": [(11,19, "RESEARCH_AREA")]}),
 ("I need to learn Big Data ", {"entities": [(16,24, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Big Data ", {"entities": [(38,46, "RESEARCH_AREA")]}),
 ("I need to do more research about Big Data ", {"entities": [(33,41, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Data Mining ", {"entities": [(32,43, "RESEARCH_AREA")]}),
 ("Who is an expert in Data Mining ", {"entities": [(20,31, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Data Mining ", {"entities": [(28,39, "RESEARCH_AREA")]}),
 ("Is there an expert for Data Mining ", {"entities": [(23,34, "RESEARCH_AREA")]}),
 ("i search an expert for Data Mining ", {"entities": [(23,34, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Data Mining ", {"entities": [(34,45, "RESEARCH_AREA")]}),
 ("Who knows the most about Data Mining ", {"entities": [(25,36, "RESEARCH_AREA")]}),
 ("who can help me with Data Mining ", {"entities": [(21,32, "RESEARCH_AREA")]}),
 ("i have a question about Data Mining ", {"entities": [(24,35, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Data Mining ", {"entities": [(34,45, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Data Mining ", {"entities": [(27,38, "RESEARCH_AREA")]}),
 ("Which professor has this research area Data Mining ", {"entities": [(39,50, "RESEARCH_AREA")]}),
 ("Who specializes in Data Mining ", {"entities": [(19,30, "RESEARCH_AREA")]}),
 ("I have a problem in Data Mining ", {"entities": [(20,31, "RESEARCH_AREA")]}),
 ("I am writing a paper about Data Mining ", {"entities": [(27,38, "RESEARCH_AREA")]}),
 ("I have a question about Data Mining ", {"entities": [(24,35, "RESEARCH_AREA")]}),
 ("I have a thesis about Data Mining ", {"entities": [(22,33, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Data Mining ", {"entities": [(28,39, "RESEARCH_AREA")]}),
 ("I have a presentation about Data Mining ", {"entities": [(28,39, "RESEARCH_AREA")]}),
 ("I need help in Data Mining ", {"entities": [(15,26, "RESEARCH_AREA")]}),
 ("I did some research in Data Mining ", {"entities": [(23,34, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Data Mining ", {"entities": [(29,40, "RESEARCH_AREA")]}),
 ("I dont understand Data Mining ", {"entities": [(18,29, "RESEARCH_AREA")]}),
 ("He is an expert in Data Mining ", {"entities": [(19,30, "RESEARCH_AREA")]}),
 ("He is a professor for Data Mining ", {"entities": [(22,33, "RESEARCH_AREA")]}),
 ("I have a test in Data Mining ", {"entities": [(17,28, "RESEARCH_AREA")]}),
 ("He teaches Data Mining ", {"entities": [(11,22, "RESEARCH_AREA")]}),
 ("I need to learn Data Mining ", {"entities": [(16,27, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Data Mining ", {"entities": [(38,49, "RESEARCH_AREA")]}),
 ("I need to do more research about Data Mining ", {"entities": [(33,44, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Data Mining ", {"entities": [(32,43, "RESEARCH_AREA")]}),
 ("Who is an expert in Data Mining ", {"entities": [(20,31, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Text Mining ", {"entities": [(32,43, "RESEARCH_AREA")]}),
 ("Who is an expert in Text Mining ", {"entities": [(20,31, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Text Mining ", {"entities": [(28,39, "RESEARCH_AREA")]}),
 ("Is there an expert for Text Mining ", {"entities": [(23,34, "RESEARCH_AREA")]}),
 ("i search an expert for Text Mining ", {"entities": [(23,34, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Text Mining ", {"entities": [(34,45, "RESEARCH_AREA")]}),
 ("Who knows the most about Text Mining ", {"entities": [(25,36, "RESEARCH_AREA")]}),
 ("who can help me with Text Mining ", {"entities": [(21,32, "RESEARCH_AREA")]}),
 ("i have a question about Text Mining ", {"entities": [(24,35, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Text Mining ", {"entities": [(34,45, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Text Mining ", {"entities": [(27,38, "RESEARCH_AREA")]}),
 ("Which professor has this research area Text Mining ", {"entities": [(39,50, "RESEARCH_AREA")]}),
 ("Who specializes in Text Mining ", {"entities": [(19,30, "RESEARCH_AREA")]}),
 ("I have a problem in Text Mining ", {"entities": [(20,31, "RESEARCH_AREA")]}),
 ("I am writing a paper about Text Mining ", {"entities": [(27,38, "RESEARCH_AREA")]}),
 ("I have a question about Text Mining ", {"entities": [(24,35, "RESEARCH_AREA")]}),
 ("I have a thesis about Text Mining ", {"entities": [(22,33, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Text Mining ", {"entities": [(28,39, "RESEARCH_AREA")]}),
 ("I have a presentation about Text Mining ", {"entities": [(28,39, "RESEARCH_AREA")]}),
 ("I need help in Text Mining ", {"entities": [(15,26, "RESEARCH_AREA")]}),
 ("I did some research in Text Mining ", {"entities": [(23,34, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Text Mining ", {"entities": [(29,40, "RESEARCH_AREA")]}),
 ("I dont understand Text Mining ", {"entities": [(18,29, "RESEARCH_AREA")]}),
 ("He is an expert in Text Mining ", {"entities": [(19,30, "RESEARCH_AREA")]}),
 ("He is a professor for Text Mining ", {"entities": [(22,33, "RESEARCH_AREA")]}),
 ("I have a test in Text Mining ", {"entities": [(17,28, "RESEARCH_AREA")]}),
 ("He teaches Text Mining ", {"entities": [(11,22, "RESEARCH_AREA")]}),
 ("I need to learn Text Mining ", {"entities": [(16,27, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Text Mining ", {"entities": [(38,49, "RESEARCH_AREA")]}),
 ("I need to do more research about Text Mining ", {"entities": [(33,44, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Text Mining ", {"entities": [(32,43, "RESEARCH_AREA")]}),
 ("Who is an expert in Text Mining ", {"entities": [(20,31, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Natural Language Processing ", {"entities": [(32,59, "RESEARCH_AREA")]}),
 ("Who is an expert in Natural Language Processing ", {"entities": [(20,47, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Natural Language Processing ", {"entities": [(28,55, "RESEARCH_AREA")]}),
 ("Is there an expert for Natural Language Processing ", {"entities": [(23,50, "RESEARCH_AREA")]}),
 ("i search an expert for Natural Language Processing ", {"entities": [(23,50, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Natural Language Processing ", {"entities": [(34,61, "RESEARCH_AREA")]}),
 ("Who knows the most about Natural Language Processing ", {"entities": [(25,52, "RESEARCH_AREA")]}),
 ("who can help me with Natural Language Processing ", {"entities": [(21,48, "RESEARCH_AREA")]}),
 ("i have a question about Natural Language Processing ", {"entities": [(24,51, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Natural Language Processing ", {"entities": [(34,61, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Natural Language Processing ", {"entities": [(27,54, "RESEARCH_AREA")]}),
 ("Which professor has this research area Natural Language Processing ", {"entities": [(39,66, "RESEARCH_AREA")]}),
 ("Who specializes in Natural Language Processing ", {"entities": [(19,46, "RESEARCH_AREA")]}),
 ("I have a problem in Natural Language Processing ", {"entities": [(20,47, "RESEARCH_AREA")]}),
 ("I am writing a paper about Natural Language Processing ", {"entities": [(27,54, "RESEARCH_AREA")]}),
 ("I have a question about Natural Language Processing ", {"entities": [(24,51, "RESEARCH_AREA")]}),
 ("I have a thesis about Natural Language Processing ", {"entities": [(22,49, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Natural Language Processing ", {"entities": [(28,55, "RESEARCH_AREA")]}),
 ("I have a presentation about Natural Language Processing ", {"entities": [(28,55, "RESEARCH_AREA")]}),
 ("I need help in Natural Language Processing ", {"entities": [(15,42, "RESEARCH_AREA")]}),
 ("I did some research in Natural Language Processing ", {"entities": [(23,50, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Natural Language Processing ", {"entities": [(29,56, "RESEARCH_AREA")]}),
 ("I dont understand Natural Language Processing ", {"entities": [(18,45, "RESEARCH_AREA")]}),
 ("He is an expert in Natural Language Processing ", {"entities": [(19,46, "RESEARCH_AREA")]}),
 ("He is a professor for Natural Language Processing ", {"entities": [(22,49, "RESEARCH_AREA")]}),
 ("I have a test in Natural Language Processing ", {"entities": [(17,44, "RESEARCH_AREA")]}),
 ("He teaches Natural Language Processing ", {"entities": [(11,38, "RESEARCH_AREA")]}),
 ("I need to learn Natural Language Processing ", {"entities": [(16,43, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Natural Language Processing ", {"entities": [(38,65, "RESEARCH_AREA")]}),
 ("I need to do more research about Natural Language Processing ", {"entities": [(33,60, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Natural Language Processing ", {"entities": [(32,59, "RESEARCH_AREA")]}),
 ("Who is an expert in Natural Language Processing ", {"entities": [(20,47, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Knowledge Graphs ", {"entities": [(32,48, "RESEARCH_AREA")]}),
 ("Who is an expert in Knowledge Graphs ", {"entities": [(20,36, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Knowledge Graphs ", {"entities": [(28,44, "RESEARCH_AREA")]}),
 ("Is there an expert for Knowledge Graphs ", {"entities": [(23,39, "RESEARCH_AREA")]}),
 ("i search an expert for Knowledge Graphs ", {"entities": [(23,39, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Knowledge Graphs ", {"entities": [(34,50, "RESEARCH_AREA")]}),
 ("Who knows the most about Knowledge Graphs ", {"entities": [(25,41, "RESEARCH_AREA")]}),
 ("who can help me with Knowledge Graphs ", {"entities": [(21,37, "RESEARCH_AREA")]}),
 ("i have a question about Knowledge Graphs ", {"entities": [(24,40, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Knowledge Graphs ", {"entities": [(34,50, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Knowledge Graphs ", {"entities": [(27,43, "RESEARCH_AREA")]}),
 ("Which professor has this research area Knowledge Graphs ", {"entities": [(39,55, "RESEARCH_AREA")]}),
 ("Who specializes in Knowledge Graphs ", {"entities": [(19,35, "RESEARCH_AREA")]}),
 ("I have a problem in Knowledge Graphs ", {"entities": [(20,36, "RESEARCH_AREA")]}),
 ("I am writing a paper about Knowledge Graphs ", {"entities": [(27,43, "RESEARCH_AREA")]}),
 ("I have a question about Knowledge Graphs ", {"entities": [(24,40, "RESEARCH_AREA")]}),
 ("I have a thesis about Knowledge Graphs ", {"entities": [(22,38, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Knowledge Graphs ", {"entities": [(28,44, "RESEARCH_AREA")]}),
 ("I have a presentation about Knowledge Graphs ", {"entities": [(28,44, "RESEARCH_AREA")]}),
 ("I need help in Knowledge Graphs ", {"entities": [(15,31, "RESEARCH_AREA")]}),
 ("I did some research in Knowledge Graphs ", {"entities": [(23,39, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Knowledge Graphs ", {"entities": [(29,45, "RESEARCH_AREA")]}),
 ("I dont understand Knowledge Graphs ", {"entities": [(18,34, "RESEARCH_AREA")]}),
 ("He is an expert in Knowledge Graphs ", {"entities": [(19,35, "RESEARCH_AREA")]}),
 ("He is a professor for Knowledge Graphs ", {"entities": [(22,38, "RESEARCH_AREA")]}),
 ("I have a test in Knowledge Graphs ", {"entities": [(17,33, "RESEARCH_AREA")]}),
 ("He teaches Knowledge Graphs ", {"entities": [(11,27, "RESEARCH_AREA")]}),
 ("I need to learn Knowledge Graphs ", {"entities": [(16,32, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Knowledge Graphs ", {"entities": [(38,54, "RESEARCH_AREA")]}),
 ("I need to do more research about Knowledge Graphs ", {"entities": [(33,49, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Knowledge Graphs ", {"entities": [(32,48, "RESEARCH_AREA")]}),
 ("Who is an expert in Knowledge Graphs ", {"entities": [(20,36, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Softwarearchitektur ", {"entities": [(32,51, "RESEARCH_AREA")]}),
 ("Who is an expert in Softwarearchitektur ", {"entities": [(20,39, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Softwarearchitektur ", {"entities": [(28,47, "RESEARCH_AREA")]}),
 ("Is there an expert for Softwarearchitektur ", {"entities": [(23,42, "RESEARCH_AREA")]}),
 ("i search an expert for Softwarearchitektur ", {"entities": [(23,42, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Softwarearchitektur ", {"entities": [(34,53, "RESEARCH_AREA")]}),
 ("Who knows the most about Softwarearchitektur ", {"entities": [(25,44, "RESEARCH_AREA")]}),
 ("who can help me with Softwarearchitektur ", {"entities": [(21,40, "RESEARCH_AREA")]}),
 ("i have a question about Softwarearchitektur ", {"entities": [(24,43, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Softwarearchitektur ", {"entities": [(34,53, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Softwarearchitektur ", {"entities": [(27,46, "RESEARCH_AREA")]}),
 ("Which professor has this research area Softwarearchitektur ", {"entities": [(39,58, "RESEARCH_AREA")]}),
 ("Who specializes in Softwarearchitektur ", {"entities": [(19,38, "RESEARCH_AREA")]}),
 ("I have a problem in Softwarearchitektur ", {"entities": [(20,39, "RESEARCH_AREA")]}),
 ("I am writing a paper about Softwarearchitektur ", {"entities": [(27,46, "RESEARCH_AREA")]}),
 ("I have a question about Softwarearchitektur ", {"entities": [(24,43, "RESEARCH_AREA")]}),
 ("I have a thesis about Softwarearchitektur ", {"entities": [(22,41, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Softwarearchitektur ", {"entities": [(28,47, "RESEARCH_AREA")]}),
 ("I have a presentation about Softwarearchitektur ", {"entities": [(28,47, "RESEARCH_AREA")]}),
 ("I need help in Softwarearchitektur ", {"entities": [(15,34, "RESEARCH_AREA")]}),
 ("I did some research in Softwarearchitektur ", {"entities": [(23,42, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Softwarearchitektur ", {"entities": [(29,48, "RESEARCH_AREA")]}),
 ("I dont understand Softwarearchitektur ", {"entities": [(18,37, "RESEARCH_AREA")]}),
 ("He is an expert in Softwarearchitektur ", {"entities": [(19,38, "RESEARCH_AREA")]}),
 ("He is a professor for Softwarearchitektur ", {"entities": [(22,41, "RESEARCH_AREA")]}),
 ("I have a test in Softwarearchitektur ", {"entities": [(17,36, "RESEARCH_AREA")]}),
 ("He teaches Softwarearchitektur ", {"entities": [(11,30, "RESEARCH_AREA")]}),
 ("I need to learn Softwarearchitektur ", {"entities": [(16,35, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Softwarearchitektur ", {"entities": [(38,57, "RESEARCH_AREA")]}),
 ("I need to do more research about Softwarearchitektur ", {"entities": [(33,52, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Softwarearchitektur ", {"entities": [(32,51, "RESEARCH_AREA")]}),
 ("Who is an expert in Softwarearchitektur ", {"entities": [(20,39, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Softwaretechnik ", {"entities": [(32,47, "RESEARCH_AREA")]}),
 ("Who is an expert in Softwaretechnik ", {"entities": [(20,35, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Softwaretechnik ", {"entities": [(28,43, "RESEARCH_AREA")]}),
 ("Is there an expert for Softwaretechnik ", {"entities": [(23,38, "RESEARCH_AREA")]}),
 ("i search an expert for Softwaretechnik ", {"entities": [(23,38, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Softwaretechnik ", {"entities": [(34,49, "RESEARCH_AREA")]}),
 ("Who knows the most about Softwaretechnik ", {"entities": [(25,40, "RESEARCH_AREA")]}),
 ("who can help me with Softwaretechnik ", {"entities": [(21,36, "RESEARCH_AREA")]}),
 ("i have a question about Softwaretechnik ", {"entities": [(24,39, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Softwaretechnik ", {"entities": [(34,49, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Softwaretechnik ", {"entities": [(27,42, "RESEARCH_AREA")]}),
 ("Which professor has this research area Softwaretechnik ", {"entities": [(39,54, "RESEARCH_AREA")]}),
 ("Who specializes in Softwaretechnik ", {"entities": [(19,34, "RESEARCH_AREA")]}),
 ("I have a problem in Softwaretechnik ", {"entities": [(20,35, "RESEARCH_AREA")]}),
 ("I am writing a paper about Softwaretechnik ", {"entities": [(27,42, "RESEARCH_AREA")]}),
 ("I have a question about Softwaretechnik ", {"entities": [(24,39, "RESEARCH_AREA")]}),
 ("I have a thesis about Softwaretechnik ", {"entities": [(22,37, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Softwaretechnik ", {"entities": [(28,43, "RESEARCH_AREA")]}),
 ("I have a presentation about Softwaretechnik ", {"entities": [(28,43, "RESEARCH_AREA")]}),
 ("I need help in Softwaretechnik ", {"entities": [(15,30, "RESEARCH_AREA")]}),
 ("I did some research in Softwaretechnik ", {"entities": [(23,38, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Softwaretechnik ", {"entities": [(29,44, "RESEARCH_AREA")]}),
 ("I dont understand Softwaretechnik ", {"entities": [(18,33, "RESEARCH_AREA")]}),
 ("He is an expert in Softwaretechnik ", {"entities": [(19,34, "RESEARCH_AREA")]}),
 ("He is a professor for Softwaretechnik ", {"entities": [(22,37, "RESEARCH_AREA")]}),
 ("I have a test in Softwaretechnik ", {"entities": [(17,32, "RESEARCH_AREA")]}),
 ("He teaches Softwaretechnik ", {"entities": [(11,26, "RESEARCH_AREA")]}),
 ("I need to learn Softwaretechnik ", {"entities": [(16,31, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Softwaretechnik ", {"entities": [(38,53, "RESEARCH_AREA")]}),
 ("I need to do more research about Softwaretechnik ", {"entities": [(33,48, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Softwaretechnik ", {"entities": [(32,47, "RESEARCH_AREA")]}),
 ("Who is an expert in Softwaretechnik ", {"entities": [(20,35, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Projektmanagement ", {"entities": [(32,49, "RESEARCH_AREA")]}),
 ("Who is an expert in Projektmanagement ", {"entities": [(20,37, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Projektmanagement ", {"entities": [(28,45, "RESEARCH_AREA")]}),
 ("Is there an expert for Projektmanagement ", {"entities": [(23,40, "RESEARCH_AREA")]}),
 ("i search an expert for Projektmanagement ", {"entities": [(23,40, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Projektmanagement ", {"entities": [(34,51, "RESEARCH_AREA")]}),
 ("Who knows the most about Projektmanagement ", {"entities": [(25,42, "RESEARCH_AREA")]}),
 ("who can help me with Projektmanagement ", {"entities": [(21,38, "RESEARCH_AREA")]}),
 ("i have a question about Projektmanagement ", {"entities": [(24,41, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Projektmanagement ", {"entities": [(34,51, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Projektmanagement ", {"entities": [(27,44, "RESEARCH_AREA")]}),
 ("Which professor has this research area Projektmanagement ", {"entities": [(39,56, "RESEARCH_AREA")]}),
 ("Who specializes in Projektmanagement ", {"entities": [(19,36, "RESEARCH_AREA")]}),
 ("I have a problem in Projektmanagement ", {"entities": [(20,37, "RESEARCH_AREA")]}),
 ("I am writing a paper about Projektmanagement ", {"entities": [(27,44, "RESEARCH_AREA")]}),
 ("I have a question about Projektmanagement ", {"entities": [(24,41, "RESEARCH_AREA")]}),
 ("I have a thesis about Projektmanagement ", {"entities": [(22,39, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Projektmanagement ", {"entities": [(28,45, "RESEARCH_AREA")]}),
 ("I have a presentation about Projektmanagement ", {"entities": [(28,45, "RESEARCH_AREA")]}),
 ("I need help in Projektmanagement ", {"entities": [(15,32, "RESEARCH_AREA")]}),
 ("I did some research in Projektmanagement ", {"entities": [(23,40, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Projektmanagement ", {"entities": [(29,46, "RESEARCH_AREA")]}),
 ("I dont understand Projektmanagement ", {"entities": [(18,35, "RESEARCH_AREA")]}),
 ("He is an expert in Projektmanagement ", {"entities": [(19,36, "RESEARCH_AREA")]}),
 ("He is a professor for Projektmanagement ", {"entities": [(22,39, "RESEARCH_AREA")]}),
 ("I have a test in Projektmanagement ", {"entities": [(17,34, "RESEARCH_AREA")]}),
 ("He teaches Projektmanagement ", {"entities": [(11,28, "RESEARCH_AREA")]}),
 ("I need to learn Projektmanagement ", {"entities": [(16,33, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Projektmanagement ", {"entities": [(38,55, "RESEARCH_AREA")]}),
 ("I need to do more research about Projektmanagement ", {"entities": [(33,50, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Projektmanagement ", {"entities": [(32,49, "RESEARCH_AREA")]}),
 ("Who is an expert in Projektmanagement ", {"entities": [(20,37, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Wissenschafts- und Forschungsmethoden ", {"entities": [(32,69, "RESEARCH_AREA")]}),
 ("Who is an expert in Wissenschafts- und Forschungsmethoden ", {"entities": [(20,57, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Wissenschafts- und Forschungsmethoden ", {"entities": [(28,65, "RESEARCH_AREA")]}),
 ("Is there an expert for Wissenschafts- und Forschungsmethoden ", {"entities": [(23,60, "RESEARCH_AREA")]}),
 ("i search an expert for Wissenschafts- und Forschungsmethoden ", {"entities": [(23,60, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Wissenschafts- und Forschungsmethoden ", {"entities": [(34,71, "RESEARCH_AREA")]}),
 ("Who knows the most about Wissenschafts- und Forschungsmethoden ", {"entities": [(25,62, "RESEARCH_AREA")]}),
 ("who can help me with Wissenschafts- und Forschungsmethoden ", {"entities": [(21,58, "RESEARCH_AREA")]}),
 ("i have a question about Wissenschafts- und Forschungsmethoden ", {"entities": [(24,61, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Wissenschafts- und Forschungsmethoden ", {"entities": [(34,71, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Wissenschafts- und Forschungsmethoden ", {"entities": [(27,64, "RESEARCH_AREA")]}),
 ("Which professor has this research area Wissenschafts- und Forschungsmethoden ", {"entities": [(39,76, "RESEARCH_AREA")]}),
 ("Who specializes in Wissenschafts- und Forschungsmethoden ", {"entities": [(19,56, "RESEARCH_AREA")]}),
 ("I have a problem in Wissenschafts- und Forschungsmethoden ", {"entities": [(20,57, "RESEARCH_AREA")]}),
 ("I am writing a paper about Wissenschafts- und Forschungsmethoden ", {"entities": [(27,64, "RESEARCH_AREA")]}),
 ("I have a question about Wissenschafts- und Forschungsmethoden ", {"entities": [(24,61, "RESEARCH_AREA")]}),
 ("I have a thesis about Wissenschafts- und Forschungsmethoden ", {"entities": [(22,59, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Wissenschafts- und Forschungsmethoden ", {"entities": [(28,65, "RESEARCH_AREA")]}),
 ("I have a presentation about Wissenschafts- und Forschungsmethoden ", {"entities": [(28,65, "RESEARCH_AREA")]}),
 ("I need help in Wissenschafts- und Forschungsmethoden ", {"entities": [(15,52, "RESEARCH_AREA")]}),
 ("I did some research in Wissenschafts- und Forschungsmethoden ", {"entities": [(23,60, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Wissenschafts- und Forschungsmethoden ", {"entities": [(29,66, "RESEARCH_AREA")]}),
 ("I dont understand Wissenschafts- und Forschungsmethoden ", {"entities": [(18,55, "RESEARCH_AREA")]}),
 ("He is an expert in Wissenschafts- und Forschungsmethoden ", {"entities": [(19,56, "RESEARCH_AREA")]}),
 ("He is a professor for Wissenschafts- und Forschungsmethoden ", {"entities": [(22,59, "RESEARCH_AREA")]}),
 ("I have a test in Wissenschafts- und Forschungsmethoden ", {"entities": [(17,54, "RESEARCH_AREA")]}),
 ("He teaches Wissenschafts- und Forschungsmethoden ", {"entities": [(11,48, "RESEARCH_AREA")]}),
 ("I need to learn Wissenschafts- und Forschungsmethoden ", {"entities": [(16,53, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Wissenschafts- und Forschungsmethoden ", {"entities": [(38,75, "RESEARCH_AREA")]}),
 ("I need to do more research about Wissenschafts- und Forschungsmethoden ", {"entities": [(33,70, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Wissenschafts- und Forschungsmethoden ", {"entities": [(32,69, "RESEARCH_AREA")]}),
 ("Who is an expert in Wissenschafts- und Forschungsmethoden ", {"entities": [(20,57, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Informationsmanagement ", {"entities": [(32,54, "RESEARCH_AREA")]}),
 ("Who is an expert in Informationsmanagement ", {"entities": [(20,42, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Informationsmanagement ", {"entities": [(28,50, "RESEARCH_AREA")]}),
 ("Is there an expert for Informationsmanagement ", {"entities": [(23,45, "RESEARCH_AREA")]}),
 ("i search an expert for Informationsmanagement ", {"entities": [(23,45, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Informationsmanagement ", {"entities": [(34,56, "RESEARCH_AREA")]}),
 ("Who knows the most about Informationsmanagement ", {"entities": [(25,47, "RESEARCH_AREA")]}),
 ("who can help me with Informationsmanagement ", {"entities": [(21,43, "RESEARCH_AREA")]}),
 ("i have a question about Informationsmanagement ", {"entities": [(24,46, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Informationsmanagement ", {"entities": [(34,56, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Informationsmanagement ", {"entities": [(27,49, "RESEARCH_AREA")]}),
 ("Which professor has this research area Informationsmanagement ", {"entities": [(39,61, "RESEARCH_AREA")]}),
 ("Who specializes in Informationsmanagement ", {"entities": [(19,41, "RESEARCH_AREA")]}),
 ("I have a problem in Informationsmanagement ", {"entities": [(20,42, "RESEARCH_AREA")]}),
 ("I am writing a paper about Informationsmanagement ", {"entities": [(27,49, "RESEARCH_AREA")]}),
 ("I have a question about Informationsmanagement ", {"entities": [(24,46, "RESEARCH_AREA")]}),
 ("I have a thesis about Informationsmanagement ", {"entities": [(22,44, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Informationsmanagement ", {"entities": [(28,50, "RESEARCH_AREA")]}),
 ("I have a presentation about Informationsmanagement ", {"entities": [(28,50, "RESEARCH_AREA")]}),
 ("I need help in Informationsmanagement ", {"entities": [(15,37, "RESEARCH_AREA")]}),
 ("I did some research in Informationsmanagement ", {"entities": [(23,45, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Informationsmanagement ", {"entities": [(29,51, "RESEARCH_AREA")]}),
 ("I dont understand Informationsmanagement ", {"entities": [(18,40, "RESEARCH_AREA")]}),
 ("He is an expert in Informationsmanagement ", {"entities": [(19,41, "RESEARCH_AREA")]}),
 ("He is a professor for Informationsmanagement ", {"entities": [(22,44, "RESEARCH_AREA")]}),
 ("I have a test in Informationsmanagement ", {"entities": [(17,39, "RESEARCH_AREA")]}),
 ("He teaches Informationsmanagement ", {"entities": [(11,33, "RESEARCH_AREA")]}),
 ("I need to learn Informationsmanagement ", {"entities": [(16,38, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Informationsmanagement ", {"entities": [(38,60, "RESEARCH_AREA")]}),
 ("I need to do more research about Informationsmanagement ", {"entities": [(33,55, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Informationsmanagement ", {"entities": [(32,54, "RESEARCH_AREA")]}),
 ("Who is an expert in Informationsmanagement ", {"entities": [(20,42, "RESEARCH_AREA")]}),
 ("Which professor is an expert in International Business ", {"entities": [(32,54, "RESEARCH_AREA")]}),
 ("Who is an expert in International Business ", {"entities": [(20,42, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in International Business ", {"entities": [(28,50, "RESEARCH_AREA")]}),
 ("Is there an expert for International Business ", {"entities": [(23,45, "RESEARCH_AREA")]}),
 ("i search an expert for International Business ", {"entities": [(23,45, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about International Business ", {"entities": [(34,56, "RESEARCH_AREA")]}),
 ("Who knows the most about International Business ", {"entities": [(25,47, "RESEARCH_AREA")]}),
 ("who can help me with International Business ", {"entities": [(21,43, "RESEARCH_AREA")]}),
 ("i have a question about International Business ", {"entities": [(24,46, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in International Business ", {"entities": [(34,56, "RESEARCH_AREA")]}),
 ("Who has the reasearch area International Business ", {"entities": [(27,49, "RESEARCH_AREA")]}),
 ("Which professor has this research area International Business ", {"entities": [(39,61, "RESEARCH_AREA")]}),
 ("Who specializes in International Business ", {"entities": [(19,41, "RESEARCH_AREA")]}),
 ("I have a problem in International Business ", {"entities": [(20,42, "RESEARCH_AREA")]}),
 ("I am writing a paper about International Business ", {"entities": [(27,49, "RESEARCH_AREA")]}),
 ("I have a question about International Business ", {"entities": [(24,46, "RESEARCH_AREA")]}),
 ("I have a thesis about International Business ", {"entities": [(22,44, "RESEARCH_AREA")]}),
 ("I am writing a thesis about International Business ", {"entities": [(28,50, "RESEARCH_AREA")]}),
 ("I have a presentation about International Business ", {"entities": [(28,50, "RESEARCH_AREA")]}),
 ("I need help in International Business ", {"entities": [(15,37, "RESEARCH_AREA")]}),
 ("I did some research in International Business ", {"entities": [(23,45, "RESEARCH_AREA")]}),
 ("I want to ask somebody about International Business ", {"entities": [(29,51, "RESEARCH_AREA")]}),
 ("I dont understand International Business ", {"entities": [(18,40, "RESEARCH_AREA")]}),
 ("He is an expert in International Business ", {"entities": [(19,41, "RESEARCH_AREA")]}),
 ("He is a professor for International Business ", {"entities": [(22,44, "RESEARCH_AREA")]}),
 ("I have a test in International Business ", {"entities": [(17,39, "RESEARCH_AREA")]}),
 ("He teaches International Business ", {"entities": [(11,33, "RESEARCH_AREA")]}),
 ("I need to learn International Business ", {"entities": [(16,38, "RESEARCH_AREA")]}),
 ("I have some questions about the topic International Business ", {"entities": [(38,60, "RESEARCH_AREA")]}),
 ("I need to do more research about International Business ", {"entities": [(33,55, "RESEARCH_AREA")]}),
 ("Which professor is an expert in International Business ", {"entities": [(32,54, "RESEARCH_AREA")]}),
 ("Who is an expert in International Business ", {"entities": [(20,42, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Informatik ", {"entities": [(32,42, "RESEARCH_AREA")]}),
 ("Who is an expert in Informatik ", {"entities": [(20,30, "RESEARCH_AREA")]}),
 ("I\m lookin for an expert in Informatik ", {"entities": [(28,38, "RESEARCH_AREA")]}),
 ("Is there an expert for Informatik ", {"entities": [(23,33, "RESEARCH_AREA")]}),
 ("i search an expert for Informatik ", {"entities": [(23,33, "RESEARCH_AREA")]}),
 ("Which professor knows a lot about Informatik ", {"entities": [(34,44, "RESEARCH_AREA")]}),
 ("Who knows the most about Informatik ", {"entities": [(25,35, "RESEARCH_AREA")]}),
 ("who can help me with Informatik ", {"entities": [(21,31, "RESEARCH_AREA")]}),
 ("i have a question about Informatik ", {"entities": [(24,34, "RESEARCH_AREA")]}),
 ("Which professor does reasearch in Informatik ", {"entities": [(34,44, "RESEARCH_AREA")]}),
 ("Who has the reasearch area Informatik ", {"entities": [(27,37, "RESEARCH_AREA")]}),
 ("Which professor has this research area Informatik ", {"entities": [(39,49, "RESEARCH_AREA")]}),
 ("Who specializes in Informatik ", {"entities": [(19,29, "RESEARCH_AREA")]}),
 ("I have a problem in Informatik ", {"entities": [(20,30, "RESEARCH_AREA")]}),
 ("I am writing a paper about Informatik ", {"entities": [(27,37, "RESEARCH_AREA")]}),
 ("I have a question about Informatik ", {"entities": [(24,34, "RESEARCH_AREA")]}),
 ("I have a thesis about Informatik ", {"entities": [(22,32, "RESEARCH_AREA")]}),
 ("I am writing a thesis about Informatik ", {"entities": [(28,38, "RESEARCH_AREA")]}),
 ("I have a presentation about Informatik ", {"entities": [(28,38, "RESEARCH_AREA")]}),
 ("I need help in Informatik ", {"entities": [(15,25, "RESEARCH_AREA")]}),
 ("I did some research in Informatik ", {"entities": [(23,33, "RESEARCH_AREA")]}),
 ("I want to ask somebody about Informatik ", {"entities": [(29,39, "RESEARCH_AREA")]}),
 ("I dont understand Informatik ", {"entities": [(18,28, "RESEARCH_AREA")]}),
 ("He is an expert in Informatik ", {"entities": [(19,29, "RESEARCH_AREA")]}),
 ("He is a professor for Informatik ", {"entities": [(22,32, "RESEARCH_AREA")]}),
 ("I have a test in Informatik ", {"entities": [(17,27, "RESEARCH_AREA")]}),
 ("He teaches Informatik ", {"entities": [(11,21, "RESEARCH_AREA")]}),
 ("I need to learn Informatik ", {"entities": [(16,26, "RESEARCH_AREA")]}),
 ("I have some questions about the topic Informatik ", {"entities": [(38,48, "RESEARCH_AREA")]}),
 ("I need to do more research about Informatik ", {"entities": [(33,43, "RESEARCH_AREA")]}),
 ("Which professor is an expert in Informatik ", {"entities": [(32,42, "RESEARCH_AREA")]}),
 ("Who is an expert in Informatik ", {"entities": [(20,30, "RESEARCH_AREA")]})]


In [ ]:
# implement function to train ner component to recognize research areas
def train_ner_research_area(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.load("en_core_web_md")
    # Getting the pipeline component
    ner = nlp.get_pipe("ner")
    
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            [text], 
                            [annotations], 
                            drop=0.2,
                            sgd=optimizer,
                            losses=losses
                )
            print(losses)
    return (nlp)
                

In [ ]:
# calling the function
nlp_research_area = train_ner_research_area(TRAIN_DATA, 30)

In [ ]:
# save custom model in file
nlp_research_area.to_disk("custom_ner_model_reseearch_area") 

### 3.9. Extraktion von Studiengängen

#### 3.9.1. Training der NER Komponente vorbereiten

**training data (hidden)**

In [28]:
TRAIN_DATA = [("Among the many users of Wirtschaftsinformatik systems are a new wave of computer scientists calling themselves data miners.", {"entities": [(24,45, "STUDY")]}),
("Among the many users of Wirtschaftsinformatik systems are a new wave of computer scientists calling themselves data miners.", {"entities": [(24,45, "STUDY")]}),
("The Digitale Transformation works the same over time.", {"entities": [(4,27, "STUDY")]}),
("Those looking for degree programs in basic academic divisions such as science, The Digitale Transformation, the humanities, or the social sciences shouldn\'t have trouble finding suitable options.", {"entities": [(83,106, "STUDY")]}),
("The student\'s graduation year impacts which Digitale Transformation classes count toward the requirements.", {"entities": [(44,67, "STUDY")]}),
("Differing tracks are tailored to students looking to pursue degrees in the liberal arts, business or Digitale Transformation and science.", {"entities": [(101,124, "STUDY")]}),
("These people are usually well trained in Digitale Transformation and in tutoring, and can help your child understand concepts that you might not be able to explain.", {"entities": [(41,64, "STUDY")]}),
("General master\'s programs cover specific subjects within Wirtschaftsinformatik - Informationsmanagement und Data Science, administration, science, math, social sciences, communications, the arts, and the humanities.", {"entities": [(57,78, "STUDY")]}),
("General master\'s programs cover specific subjects within Wirtschaftsinformatik - Informationsmanagement und Data Science, administration, science, math, social sciences, communications, the arts, and the humanities.", {"entities": [(57,120, "STUDY")]}),
("General master\'s programs cover specific subjects within Wirtschaftsinformatik - Informationsmanagement und Data Science, administration, science, math, social sciences, communications, the arts, and the humanities.", {"entities": [(81,120, "STUDY")]}),
("Among the specific courses offered are options in Wirtschaftsinformatik - Informationsmanagement und Data Science, finite math, calculus, astronomy, geology, classics, and political science.", {"entities": [(50,71, "STUDY")]}),
("Among the specific courses offered are options in Wirtschaftsinformatik - Informationsmanagement und Data Science, finite math, calculus, astronomy, geology, classics, and political science.", {"entities": [(50,113, "STUDY")]}),
("Among the specific courses offered are options in Wirtschaftsinformatik - Informationsmanagement und Data Science, finite math, calculus, astronomy, geology, classics, and political science.", {"entities": [(74,113, "STUDY")]}),
("More than thirty percent of students that reside in the state of Colorado who are interested in pursuing degrees in Wirtschaftsinformatik - Informationsmanagement und Data Science, math, engineering, and technology do so through their home university at Colorado State University.", {"entities": [(116,137, "STUDY")]}),
("More than thirty percent of students that reside in the state of Colorado who are interested in pursuing degrees in Wirtschaftsinformatik - Informationsmanagement und Data Science, math, engineering, and technology do so through their home university at Colorado State University.", {"entities": [(116,179, "STUDY")]}),
("More than thirty percent of students that reside in the state of Colorado who are interested in pursuing degrees in Wirtschaftsinformatik - Informationsmanagement und Data Science, math, engineering, and technology do so through their home university at Colorado State University.", {"entities": [(140,179, "STUDY")]}),
("It takes most of the International Business out of the process as well as the guess work.", {"entities": [(21,43, "STUDY")]}),
("The National Gardening Association\'s mulch calculator makes you do little bit of International Business.", {"entities": [(81,103, "STUDY")]}),
("If that\'s too much International Business for you, go back to the handy online mulch calculators.", {"entities": [(19,41, "STUDY")]}),
("This plan is for a frame roughly four feet by two and a half feet, but with a little International Business you can easily adapt it to fit whatever size window you happen to have.", {"entities": [(85,107, "STUDY")]}),
("Notable people of the past and present are featured in captivating human interest stories ranging from Robert DeNiro to a professor campaigning for better Wirtschaftsinformatik and science education in schools.", {"entities": [(155,176, "STUDY")]}),
("Humans invent abstract Wirtschaftsinformatik Analytics, basically making it up out of their imaginations, yet math magically turns out to describe the world.", {"entities": [(23,44, "STUDY")]}),
("He\'s not an obvious choice for Macbeth, tho, looking more like a slightly seedy Wirtschaftsinformatik Analytics professor than a military hero.", {"entities": [(80,101, "STUDY")]}),
("Join our Software Engineering on the Web Mailing List to stay up-to-date on Math on the Web.", {"entities": [(9,29, "STUDY")]}),
("He \'s not an obvious choice for Macbeth, tho, looking more like a slightly seedy Software Engineering professor than a military hero.", {"entities": [(81,101, "STUDY")]}),
("Handwrite a Software Engineering equation, and then convert it into a neatly typewritten image to paste into a report or a presentation.", {"entities": [(12,32, "STUDY")]}),
("Hannah was confused by the asterisk in her Software Engineering homework, because she didn\'t know it meant to multiply.", {"entities": [(43,63, "STUDY")]}),
("Explore Learning-With over  interactive math and Software Engineering activities available online, the Explore Learning site is an excellent resource for kids who need help with math and science topics, or who just love learning.", {"entities": [(49,69, "STUDY")]}),
("Software Engineering Tutor-Teachers and students will find this website useful for help with writing math expressions and algebra word problems.", {"entities": [(0,20, "STUDY")]}),
("In addition, students can ask specific questions on the Software Engineering help message board.", {"entities": [(56,76, "STUDY")]}),
("Management und Vertrieb practice lessons are available for Pre-K through eighth grade and include algebra as well.", {"entities": [(0,23, "STUDY")]}),
("If you see that your child\'s Management und Vertrieb scores are dropping, and you\'re worried than neither you nor your child\'s teacher seems to be able to provide the right support, hiring a math tutor can be a great option.", {"entities": [(29,52, "STUDY")]}),
("Some schools prohibit extremely short teen mini skirts, so check your school\'s dress code before donning one to Management und Vertrieb class.", {"entities": [(112,135, "STUDY")]}),
("Why do girls lose interest in Management und Vertrieb and science so easily?.", {"entities": [(30,53, "STUDY")]}),
("To Mr. Loomis, International Tourism Management teacher minus a heart.", {"entities": [(15,47, "STUDY")]}),
("International Tourism Management .com has tutorials and plans to help make math simpler.", {"entities": [(0,32, "STUDY")]}),
("core courses include Intercultural Management, English, social studies, and science.", {"entities": [(21,45, "STUDY")]}),
("Teen twins can help one another with what they are going through, whether it is a difficult personal problem or something more tangible like Intercultural Management homework.", {"entities": [(141,165, "STUDY")]}),
("Taking a business management or Weinmarketing und Management class could help you later on in life if you ever own your own business or help run a business with a friend or boss.", {"entities": [(32,60, "STUDY")]}),
("There are many different business and industry classes that you can take in high school, ranging from Weinmarketing und Management to cooking classes.", {"entities": [(102,130, "STUDY")]}),
("A Betriebswirtschaft class, for example will not only help you with your basic math skills, but it will also help you learn how to budget your money wisely and how to manage money better as an adult.", {"entities": [(2,20, "STUDY")]}),
("For just a small snapshot of student achievement at the college, the current range of SAT verbal and Betriebswirtschaft scores for enrolled freshmen is  to  out of  possible points.", {"entities": [(101,119, "STUDY")]}),
("Courses are available in world languages, Digitale Transformation, writing, fine arts, theatre, math, natural sciences, anthropology, education, and political science, among others.", {"entities": [(42,65, "STUDY")]}),
("The test measures levels in core skills of Digitale Transformation, writing, reading, and math, and is available for ESL students as well.", {"entities": [(43,66, "STUDY")]}),
("Test topics range from Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend and memory to concentration and visual tests.", {"entities": [(23,44, "STUDY")]}),
("Test topics range from Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend and memory to concentration and visual tests.", {"entities": [(23,88, "STUDY")]}),
("Test topics range from Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend and memory to concentration and visual tests.", {"entities": [(47,70, "STUDY")]}),
("Products are available for PC and for every major video game console, teaching topics including reading, math, science, Hochschulleitung and social studies.", {"entities": [(120,136, "STUDY")]}),
("For older children, educational games get more elaborate, allowing them to solve problems, try virtual science experiments, and test out careers as well as learning the basics like reading and Hochschulleitung .", {"entities": [(193,209, "STUDY")]}),
("If your son loves cars, he will probably be more excited about learning if you get the racing version of the Hochschulleitung .", {"entities": [(109,125, "STUDY")]}),
("International Business. It\'s a subject that many people dread well into adulthood, but simple arithmetic is absolutely fundamental to proper functioning in today\'s society.", {"entities": [(0,22, "STUDY")]}),
("One of the simpler, yet addictive free educational video games to play online that focuses on numbers is International Business.", {"entities": [(105,127, "STUDY")]}),
("Hochschulleitung Drills - This site offers not only fraction worksheets but printable manipulatives to help your child learn fractions hands on.", {"entities": [(0,16, "STUDY")]}),
("These games make learning fun, and there is a game to fit every basic Wirtschaft und Verkehr concept.", {"entities": [(70,92, "STUDY")]}),
("you don\'t have to purchase games to have fun with Wirtschaft und Verkehr.", {"entities": [(50,72, "STUDY")]}),
("An emphasis on reading, writing, and Wirtschaft und Verkehr is necessary to prepare children for academic success.", {"entities": [(37,59, "STUDY")]}),
("Children can learn Wirtschaft und Verkehr in many ways, not just in a classroom setting.", {"entities": [(19,41, "STUDY")]}),
("Many games require some form of reading, sorting, early Wirtschaft und Verkehr or problem-solving.", {"entities": [(56,78, "STUDY")]}),
("If you can find cool Wirtschaft und Verkehr games for kids, you can turn what is sometimes a boring subject into an exciting experience that they will look forward to over and over.", {"entities": [(21,43, "STUDY")]}),
("The complicated array of Weinmarketing und Management problems intimidated me.", {"entities": [(25,53, "STUDY")]}),
("The students could not figure out the Weinmarketing und Management problem until their teacher provided them with a helpful formula.", {"entities": [(38,66, "STUDY")]}),
("He calculated the answer to the Weinmarketing und Management problems using the formulae provided by the teacher.", {"entities": [(32,60, "STUDY")]}),
("Luckily, when a Weinmarketing und Management problem seems insurmountable, I can call my brother Billy to help.", {"entities": [(16,44, "STUDY")]}),
("The program includes a strong foundation in language development, early literacy, and early Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(92,113, "STUDY")]}),
("The program includes a strong foundation in language development, early literacy, and early Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(92,155, "STUDY")]}),
("The program includes a strong foundation in language development, early literacy, and early Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(116,155, "STUDY")]}),
("Studies of abused children\'s intellectual performance find lower scores in both verbal and Wirtschaftsinformatik - Informationsmanagement und Data Science and visual-spatial areas.", {"entities": [(91,112, "STUDY")]}),
("Studies of abused children\'s intellectual performance find lower scores in both verbal and Wirtschaftsinformatik - Informationsmanagement und Data Science and visual-spatial areas.", {"entities": [(91,154, "STUDY")]}),
("Studies of abused children\'s intellectual performance find lower scores in both verbal and Wirtschaftsinformatik - Informationsmanagement und Data Science and visual-spatial areas.", {"entities": [(115,154, "STUDY")]}),
("These include correspondence courses, full curricula, and single topic books in areas such as Wirtschaftsinformatik - Informationsmanagement und Data Science or phonics.He calculated the answer to the Wirtschaftsinformatik - Informationsmanagement und Data Science problems using the formulae provided by the teacher.", {"entities": [(94,115, "STUDY")]}),
("These include correspondence courses, full curricula, and single topic books in areas such as Wirtschaftsinformatik - Informationsmanagement und Data Science or phonics.He calculated the answer to the Wirtschaftsinformatik - Informationsmanagement und Data Science problems using the formulae provided by the teacher.", {"entities": [(94,157, "STUDY")]}),
("These include correspondence courses, full curricula, and single topic books in areas such as Wirtschaftsinformatik - Informationsmanagement und Data Science or phonics.He calculated the answer to the Wirtschaftsinformatik - Informationsmanagement und Data Science problems using the formulae provided by the teacher.", {"entities": [(118,157, "STUDY")]}),
("Luckily, when a Wirtschaftsinformatik - Informationsmanagement und Data Science problem seems insurmountable, I can call my brother Billy to help.", {"entities": [(16,37, "STUDY")]}),
("Luckily, when a Wirtschaftsinformatik - Informationsmanagement und Data Science problem seems insurmountable, I can call my brother Billy to help.", {"entities": [(16,79, "STUDY")]}),
("Luckily, when a Wirtschaftsinformatik - Informationsmanagement und Data Science problem seems insurmountable, I can call my brother Billy to help.", {"entities": [(40,79, "STUDY")]}),
(" Wirtschaftsinformatik - Informationsmanagement und Data Science, polygon and CPU performance make previously impossible gaming into reality.", {"entities": [(1,22, "STUDY")]}),
(" Wirtschaftsinformatik - Informationsmanagement und Data Science, polygon and CPU performance make previously impossible gaming into reality.", {"entities": [(1,64, "STUDY")]}),
(" Wirtschaftsinformatik - Informationsmanagement und Data Science, polygon and CPU performance make previously impossible gaming into reality.", {"entities": [(25,64, "STUDY")]}),
("The app library for each phone also has academic uses, with apps available for Wirtschaftsinformatik - Informationsmanagement und Data Science calculations, scientific equations, history facts and trivia quizzes.", {"entities": [(79,100, "STUDY")]}),
("The app library for each phone also has academic uses, with apps available for Wirtschaftsinformatik - Informationsmanagement und Data Science calculations, scientific equations, history facts and trivia quizzes.", {"entities": [(79,142, "STUDY")]}),
("The app library for each phone also has academic uses, with apps available for Wirtschaftsinformatik - Informationsmanagement und Data Science calculations, scientific equations, history facts and trivia quizzes.", {"entities": [(103,142, "STUDY")]}),
("The program includes a strong foundation in language development, early literacy, and early Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(92,113, "STUDY")]}),
("The program includes a strong foundation in language development, early literacy, and early Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(92,155, "STUDY")]}),
("The program includes a strong foundation in language development, early literacy, and early Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(116,155, "STUDY")]}),
("Studies of abused children\'s intellectual performance find lower scores in both verbal and Wirtschaftsinformatik - Informationsmanagement und Data Science and visual-spatial areas.", {"entities": [(91,112, "STUDY")]}),
("Studies of abused children\'s intellectual performance find lower scores in both verbal and Wirtschaftsinformatik - Informationsmanagement und Data Science and visual-spatial areas.", {"entities": [(91,154, "STUDY")]}),
("Studies of abused children\'s intellectual performance find lower scores in both verbal and Wirtschaftsinformatik - Informationsmanagement und Data Science and visual-spatial areas.", {"entities": [(115,154, "STUDY")]}),
("These include correspondence courses, full curricula, and single topic books in areas such as Wirtschaftsinformatik - Informationsmanagement und Data Science or phonics.Children can learn Wirtschaftsinformatik - Informationsmanagement und Data Science in many ways, not just in a classroom setting.", {"entities": [(94,115, "STUDY")]}),
("These include correspondence courses, full curricula, and single topic books in areas such as Wirtschaftsinformatik - Informationsmanagement und Data Science or phonics.Children can learn Wirtschaftsinformatik - Informationsmanagement und Data Science in many ways, not just in a classroom setting.", {"entities": [(94,157, "STUDY")]}),
("These include correspondence courses, full curricula, and single topic books in areas such as Wirtschaftsinformatik - Informationsmanagement und Data Science or phonics.Children can learn Wirtschaftsinformatik - Informationsmanagement und Data Science in many ways, not just in a classroom setting.", {"entities": [(118,157, "STUDY")]}),
("Many games require some form of reading, sorting, early Wirtschaftsinformatik - Informationsmanagement und Data Science or problem-solving.", {"entities": [(56,77, "STUDY")]}),
("Many games require some form of reading, sorting, early Wirtschaftsinformatik - Informationsmanagement und Data Science or problem-solving.", {"entities": [(56,119, "STUDY")]}),
("Many games require some form of reading, sorting, early Wirtschaftsinformatik - Informationsmanagement und Data Science or problem-solving.", {"entities": [(80,119, "STUDY")]}),
("If you can find cool Wirtschaftsinformatik - Informationsmanagement und Data Science games for kids, you can turn what is sometimes a boring subject into an exciting experience that they will look forward to over and over.", {"entities": [(21,42, "STUDY")]}),
("If you can find cool Wirtschaftsinformatik - Informationsmanagement und Data Science games for kids, you can turn what is sometimes a boring subject into an exciting experience that they will look forward to over and over.", {"entities": [(21,84, "STUDY")]}),
("If you can find cool Wirtschaftsinformatik - Informationsmanagement und Data Science games for kids, you can turn what is sometimes a boring subject into an exciting experience that they will look forward to over and over.", {"entities": [(45,84, "STUDY")]}),
("The complicated array of Wirtschaftsinformatik - Informationsmanagement und Data Science problems intimidated me.", {"entities": [(25,46, "STUDY")]}),
("The complicated array of Wirtschaftsinformatik - Informationsmanagement und Data Science problems intimidated me.", {"entities": [(25,88, "STUDY")]}),
("The complicated array of Wirtschaftsinformatik - Informationsmanagement und Data Science problems intimidated me.", {"entities": [(49,88, "STUDY")]}),
("The students could not figure out the Wirtschaftsinformatik - Informationsmanagement und Data Science problem until their teacher provided them with a helpful formula.", {"entities": [(38,59, "STUDY")]}),
("The students could not figure out the Wirtschaftsinformatik - Informationsmanagement und Data Science problem until their teacher provided them with a helpful formula.", {"entities": [(38,101, "STUDY")]}),
("The students could not figure out the Wirtschaftsinformatik - Informationsmanagement und Data Science problem until their teacher provided them with a helpful formula.", {"entities": [(62,101, "STUDY")]}),
("He calculated the answer to the Wirtschaftsinformatik - Informationsmanagement und Data Science problems using the formulae provided by the teacher.", {"entities": [(32,53, "STUDY")]}),
("He calculated the answer to the Wirtschaftsinformatik - Informationsmanagement und Data Science problems using the formulae provided by the teacher.", {"entities": [(32,95, "STUDY")]}),
("He calculated the answer to the Wirtschaftsinformatik - Informationsmanagement und Data Science problems using the formulae provided by the teacher.", {"entities": [(56,95, "STUDY")]}),
("Luckily, when a Wirtschaftsinformatik - Informationsmanagement und Data Science problem seems insurmountable, I can call my brother Billy to help.", {"entities": [(16,37, "STUDY")]}),
("Luckily, when a Wirtschaftsinformatik - Informationsmanagement und Data Science problem seems insurmountable, I can call my brother Billy to help.", {"entities": [(16,79, "STUDY")]}),
("Luckily, when a Wirtschaftsinformatik - Informationsmanagement und Data Science problem seems insurmountable, I can call my brother Billy to help.", {"entities": [(40,79, "STUDY")]}),
("Does any section of science consider time Wirtschaftsinformatik a possibility? .", {"entities": [(42,63, "STUDY")]}),
("Groups of people will do Wirtschaftsinformatik this same way.", {"entities": [(25,46, "STUDY")]}),
("The ability of Wirtschaftsinformatik and technology to improve human life is known to us.This is not Wirtschaft und Verkehr.", {"entities": [(15,36, "STUDY")]}),
("The ability of Wirtschaftsinformatik and technology to improve human life is known to us.This is not Wirtschaft und Verkehr.", {"entities": [(101,123, "STUDY")]}),
("When Wirtschaftsinformatik is in your clothes, medicine, eyeglasses, wallet, tires, walls, makeup, jewelry, cookware, tennis shoes, binoculars, and everything else you own, those things will do more than you can imagine—the stuff of science fiction.", {"entities": [(5,26, "STUDY")]}),
("Wirtschaftsinformatik has been of priceless value to chemists, but it has more than once happened in the history of science that a hypothesis, after having been useful in the discovery Present and the co-ordination of knowledge, has been aban- position doned and replaced by one more in harmony with later of the discoveries.", {"entities": [(0,21, "STUDY")]}),
("His residence was generally divided into two parts - one his workshop for Wirtschaft und Verkehr, the other his reception-room for society.", {"entities": [(74,96, "STUDY")]}),
("With a penchant for Wirtschaftsinformatik and science and my magic, I can cure what others could not.A co-ordinate woman\'s college, the William Smith school for women, opened in , was endowed in  by William Smith of Geneva, who at the same time provided for a Hall of Science and for further instruction in Wirtschaft und Verkehr, especially in biology and psychology. .", {"entities": [(20,41, "STUDY")]}),
("With a penchant for Wirtschaftsinformatik and science and my magic, I can cure what others could not.A co-ordinate woman\'s college, the William Smith school for women, opened in , was endowed in  by William Smith of Geneva, who at the same time provided for a Hall of Science and for further instruction in Wirtschaft und Verkehr, especially in biology and psychology. .", {"entities": [(307,329, "STUDY")]}),
("This contradiction arises from the fact that military Wirtschaftsinformatik assumes the strength of an army to be identical with its numbers. .", {"entities": [(54,75, "STUDY")]}),
("First there were Wirtschaft und Verkehr, themselves only just emerging from a confused conception of their true method; especially those which studied the borderland of physical and mental phenomena, the medical sciences; and pre-eminently that science which has since become so popular, the science of biology.", {"entities": [(17,39, "STUDY")]}),
("It also becomes clear that only where such mental life really appears need we assign an independent existence, but that the purposes of everyday life as well as those of Wirtschaftsinformatik are equally served if we deprive the material things outside of us of an independence, and assign to them merely a connected existence through the universal substance by the action of which alone they can appear to us.", {"entities": [(170,191, "STUDY")]}),
("Randy, a Wirtschaftsinformatik buff, was in the top third of the grad\xaduates and was also his class president.", {"entities": [(9,30, "STUDY")]}),
("Though cases like these are not really how the Wirtschaftsinformatik will be used, they illustrate the principle.", {"entities": [(47,68, "STUDY")]}),
("They didn\'t know about Wirtschaftsinformatik back then - but then, they didn\'t know about cesarean delivery either.", {"entities": [(23,44, "STUDY")]}),
("When Lotze published these works, Wirtschaftsinformatik was still much under the influence of Schelling\'s philosophy of nature.", {"entities": [(34,55, "STUDY")]}),
("Vico was the first thinker who asked, Why have we Wirtschaftsinformatik .", {"entities": [(50,71, "STUDY")]}),
("Others contend, and feel they have Wirtschaftsinformatik to support, that humans can live beyond five hundred.", {"entities": [(35,56, "STUDY")]}),
("He looked at the slight Natural with dark hair and eyes who happened to have a doctorate in every type of Wirtschaftsinformatik he could name.", {"entities": [(106,127, "STUDY")]}),
("Well, go on, he continued, returning to his hobby; tell me how the Germans have taught you can be an expert in Wirtschaft und Verkehr.", {"entities": [(111,133, "STUDY")]}),
("From the standpoint from which the Wirtschaftsinformatik of history now regards its subject on the path it now follows, seeking the causes of events in man\'s freewill, a scientific enunciation of those laws is impossible, for however man\'s free will may be restricted, as soon as we recognize it as a force not subject to law, the existence of law becomes impossible.", {"entities": [(35,56, "STUDY")]}),
("The theory has not found general acceptance, but it proved of great value to Wirtschaft und Verkehr, owing to the extensive additions to the knowledge of the structure of mountain ranges which its author made in endeavouring to find facts to support it.", {"entities": [(77,99, "STUDY")]}),
("The long series of memoirs - some of them complete treatises of great moment in the history of Wirtschaftsinformatik - communicated by Lagrange to the Berlin Academy between the years  and  were not the only fruits of his exile.", {"entities": [(95,116, "STUDY")]}),
("To Lagrange, perhaps more than to any other, the theory of differential equations is indebted for its position as a Wirtschaft und Verkehr, rather than a collection of ingenious artifices for the solution of particular problems. To the calculus of finite differences he contributed the beautiful formula of interpolation which bears his name; although substantially the same result seems to have been previously obtained by Euler.", {"entities": [(116,138, "STUDY")]}),
("In Aristotle the Xl yos of a thing is its definition, including its formal cause, while the ultimate principles of a Wirtschaftsinformatik are apxal, the reasons (in a common modern sense) which explain all its particular facts.", {"entities": [(117,138, "STUDY")]}),
("In , having graduated as doctor of Wirtschaftsinformatik and doctor of science, he was appointed to organize the new faculty of science at Besancon, where he acted as dean and professor of chemistry from  to .", {"entities": [(35,56, "STUDY")]}),
("In Wirtschaftsinformatik - Informationsmanagement und Data Science Brazil has accomplished very little, although many eminent foreign naturalists have spent years of study within her borders.", {"entities": [(3,24, "STUDY")]}),
("In Wirtschaftsinformatik - Informationsmanagement und Data Science Brazil has accomplished very little, although many eminent foreign naturalists have spent years of study within her borders.", {"entities": [(3,66, "STUDY")]}),
("In Wirtschaftsinformatik - Informationsmanagement und Data Science Brazil has accomplished very little, although many eminent foreign naturalists have spent years of study within her borders.", {"entities": [(27,66, "STUDY")]}),
("These explorations cover every branch Wirtschaftsinformatik - Informationsmanagement und Data Science and resulted in publications of inestimable scientific value.", {"entities": [(38,59, "STUDY")]}),
("These explorations cover every branch Wirtschaftsinformatik - Informationsmanagement und Data Science and resulted in publications of inestimable scientific value.", {"entities": [(38,101, "STUDY")]}),
("These explorations cover every branch Wirtschaftsinformatik - Informationsmanagement und Data Science and resulted in publications of inestimable scientific value.", {"entities": [(62,101, "STUDY")]}),
("Wirtschaftsinformatik - Informationsmanagement und Data Science would solve everything, prosperity would grow indefinitely, and people would thrive.Dr. Bell is proficient in many fields of Wirtschaftsinformatik - Informationsmanagement und Data Science , and has the art of making every subject he touches interesting, even the most abstruse theories.", {"entities": [(0,21, "STUDY")]}),
("Wirtschaftsinformatik - Informationsmanagement und Data Science would solve everything, prosperity would grow indefinitely, and people would thrive.Dr. Bell is proficient in many fields of Wirtschaftsinformatik - Informationsmanagement und Data Science , and has the art of making every subject he touches interesting, even the most abstruse theories.", {"entities": [(0,63, "STUDY")]}),
("Wirtschaftsinformatik - Informationsmanagement und Data Science would solve everything, prosperity would grow indefinitely, and people would thrive.Dr. Bell is proficient in many fields of Wirtschaftsinformatik - Informationsmanagement und Data Science , and has the art of making every subject he touches interesting, even the most abstruse theories.", {"entities": [(24,63, "STUDY")]}),
("We are most interested when Wirtschaftsinformatik - Informationsmanagement und Data Science reports what those men already know practically or instinctively, for that alone is a true humanity, or account of human experience.", {"entities": [(28,49, "STUDY")]}),
("We are most interested when Wirtschaftsinformatik - Informationsmanagement und Data Science reports what those men already know practically or instinctively, for that alone is a true humanity, or account of human experience.", {"entities": [(28,91, "STUDY")]}),
("We are most interested when Wirtschaftsinformatik - Informationsmanagement und Data Science reports what those men already know practically or instinctively, for that alone is a true humanity, or account of human experience.", {"entities": [(52,91, "STUDY")]}),
("Wirtschaftsinformatik - Informationsmanagement und Data Science, seeing in history innumerable instances of the fact that the size of any army does not coincide with its strength and that small detachments defeat larger ones, obscurely admits the existence of this unknown factor and tries to discover it--now in a geometric formation, now in the equipment employed, now, and most usually, in the genius of the commanders.", {"entities": [(0,21, "STUDY")]}),
("Wirtschaftsinformatik - Informationsmanagement und Data Science, seeing in history innumerable instances of the fact that the size of any army does not coincide with its strength and that small detachments defeat larger ones, obscurely admits the existence of this unknown factor and tries to discover it--now in a geometric formation, now in the equipment employed, now, and most usually, in the genius of the commanders.", {"entities": [(0,63, "STUDY")]}),
("Wirtschaftsinformatik - Informationsmanagement und Data Science, seeing in history innumerable instances of the fact that the size of any army does not coincide with its strength and that small detachments defeat larger ones, obscurely admits the existence of this unknown factor and tries to discover it--now in a geometric formation, now in the equipment employed, now, and most usually, in the genius of the commanders.", {"entities": [(24,63, "STUDY")]}),
("Gabriel didn\'t need to understand Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(34,55, "STUDY")]}),
("Gabriel didn\'t need to understand Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(34,97, "STUDY")]}),
("Gabriel didn\'t need to understand Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(58,97, "STUDY")]}),
("He became teacher of Wirtschaftsinformatik - Informationsmanagement und Data Science successively at the French gymnasium in Berlin, and at the military academy, and on the foundation of the university of Berlin in  he was chosen professor of physics.", {"entities": [(21,42, "STUDY")]}),
("He became teacher of Wirtschaftsinformatik - Informationsmanagement und Data Science successively at the French gymnasium in Berlin, and at the military academy, and on the foundation of the university of Berlin in  he was chosen professor of physics.", {"entities": [(21,84, "STUDY")]}),
("He became teacher of Wirtschaftsinformatik - Informationsmanagement und Data Science successively at the French gymnasium in Berlin, and at the military academy, and on the foundation of the university of Berlin in  he was chosen professor of physics.", {"entities": [(45,84, "STUDY")]}),
("Wirtschaftsinformatik - Informationsmanagement und Data Science, he says, may be compared to a tree; metaphysics is the root, physics is the trunk, and the three chief branches are mechanics, medicine and Ouvres, viii.", {"entities": [(0,21, "STUDY")]}),
("Wirtschaftsinformatik - Informationsmanagement und Data Science, he says, may be compared to a tree; metaphysics is the root, physics is the trunk, and the three chief branches are mechanics, medicine and Ouvres, viii.", {"entities": [(0,63, "STUDY")]}),
("Wirtschaftsinformatik - Informationsmanagement und Data Science, he says, may be compared to a tree; metaphysics is the root, physics is the trunk, and the three chief branches are mechanics, medicine and Ouvres, viii.", {"entities": [(24,63, "STUDY")]}),
("The book will contain four essays, all in French, with the general title of Project of Wirtschaftsinformatik - Informationsmanagement und Data Science, capable of raising our nature to its highest perfection; also Dioptrics, Meteors and “Geometry, wherein the most curious matters which the author could select as a proof of the universal science which he proposes are explained in such a way that even the unlearned may understand them.", {"entities": [(87,108, "STUDY")]}),
("The book will contain four essays, all in French, with the general title of Project of Wirtschaftsinformatik - Informationsmanagement und Data Science, capable of raising our nature to its highest perfection; also Dioptrics, Meteors and “Geometry, wherein the most curious matters which the author could select as a proof of the universal science which he proposes are explained in such a way that even the unlearned may understand them.", {"entities": [(87,150, "STUDY")]}),
("The book will contain four essays, all in French, with the general title of Project of Wirtschaftsinformatik - Informationsmanagement und Data Science, capable of raising our nature to its highest perfection; also Dioptrics, Meteors and “Geometry, wherein the most curious matters which the author could select as a proof of the universal science which he proposes are explained in such a way that even the unlearned may understand them.", {"entities": [(111,150, "STUDY")]}),
("The absurd attempt was, and sometimes is still, made by geographers to include all natural science in geography; but it is more common for specialists in the various detailed Wirtschaftsinformatik - Informationsmanagement und Data Science to think, and sometimes to assert, that the ground of physical geography is now fully occupied by these sciences.", {"entities": [(175,196, "STUDY")]}),
("The absurd attempt was, and sometimes is still, made by geographers to include all natural science in geography; but it is more common for specialists in the various detailed Wirtschaftsinformatik - Informationsmanagement und Data Science to think, and sometimes to assert, that the ground of physical geography is now fully occupied by these sciences.", {"entities": [(175,238, "STUDY")]}),
("The absurd attempt was, and sometimes is still, made by geographers to include all natural science in geography; but it is more common for specialists in the various detailed Wirtschaftsinformatik - Informationsmanagement und Data Science to think, and sometimes to assert, that the ground of physical geography is now fully occupied by these sciences.", {"entities": [(199,238, "STUDY")]}),
("Of the former, the first, published in , was on the dynamics of a particle; and afterwards there followed a number of concise treatises on thermodynamics, heat, light, properties of matter and dynamics, together with an admirably lucid volume of popular lectures on Recent Advances in Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(285,306, "STUDY")]}),
("Of the former, the first, published in , was on the dynamics of a particle; and afterwards there followed a number of concise treatises on thermodynamics, heat, light, properties of matter and dynamics, together with an admirably lucid volume of popular lectures on Recent Advances in Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(285,348, "STUDY")]}),
("Of the former, the first, published in , was on the dynamics of a particle; and afterwards there followed a number of concise treatises on thermodynamics, heat, light, properties of matter and dynamics, together with an admirably lucid volume of popular lectures on Recent Advances in Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(309,348, "STUDY")]}),
("In one sense there is no Wirtschaftsinformatik - Informationsmanagement und Data Science of applied mathematics.", {"entities": [(25,46, "STUDY")]}),
("In one sense there is no Wirtschaftsinformatik - Informationsmanagement und Data Science of applied mathematics.", {"entities": [(25,88, "STUDY")]}),
("In one sense there is no Wirtschaftsinformatik - Informationsmanagement und Data Science of applied mathematics.", {"entities": [(49,88, "STUDY")]}),
("By Ancillon he was grounded in religion, in history and Wirtschaftsinformatik - Informationsmanagement und Data Science, his natural taste for the antique and the picturesque making it easy for his tutor to impress upon him his own hatred of the Revolution and its principles.", {"entities": [(56,77, "STUDY")]}),
("By Ancillon he was grounded in religion, in history and Wirtschaftsinformatik - Informationsmanagement und Data Science, his natural taste for the antique and the picturesque making it easy for his tutor to impress upon him his own hatred of the Revolution and its principles.", {"entities": [(56,119, "STUDY")]}),
("By Ancillon he was grounded in religion, in history and Wirtschaftsinformatik - Informationsmanagement und Data Science, his natural taste for the antique and the picturesque making it easy for his tutor to impress upon him his own hatred of the Revolution and its principles.", {"entities": [(80,119, "STUDY")]}),
("Idem, Degeneration, a Chapter in Darwinism, reprinted in the Advancement of Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(76,97, "STUDY")]}),
("Idem, Degeneration, a Chapter in Darwinism, reprinted in the Advancement of Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(76,139, "STUDY")]}),
("Idem, Degeneration, a Chapter in Darwinism, reprinted in the Advancement of Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(100,139, "STUDY")]}),
("In spite of his almost incessant controversies with the Aristotelians, he found time to make a comprehensive study of contemporary Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(131,152, "STUDY")]}),
("In spite of his almost incessant controversies with the Aristotelians, he found time to make a comprehensive study of contemporary Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(131,194, "STUDY")]}),
("In spite of his almost incessant controversies with the Aristotelians, he found time to make a comprehensive study of contemporary Wirtschaftsinformatik - Informationsmanagement und Data Science.", {"entities": [(155,194, "STUDY")]}),
("The Royal Scottish Museum, structurally united to the university, contains collections illustrative of industry, art, Wirtschaftsinformatik - Informationsmanagement und Data Science and natural history; and Minto House college and Heriot-Watt college are practically adjuncts of the university.", {"entities": [(118,139, "STUDY")]}),
("The Royal Scottish Museum, structurally united to the university, contains collections illustrative of industry, art, Wirtschaftsinformatik - Informationsmanagement und Data Science and natural history; and Minto House college and Heriot-Watt college are practically adjuncts of the university.", {"entities": [(118,181, "STUDY")]}),
("The Royal Scottish Museum, structurally united to the university, contains collections illustrative of industry, art, Wirtschaftsinformatik - Informationsmanagement und Data Science and natural history; and Minto House college and Heriot-Watt college are practically adjuncts of the university.", {"entities": [(142,181, "STUDY")]}),
("Its graduates also give lectures on the various branches of medicine and Wirtschaftsinformatik - Informationsmanagement und Data Science requisite for the degree of doctor of medicine, and those extra-academical courses are recognized, under certain restrictions, by the University Court, as qualifying for the degree.", {"entities": [(73,94, "STUDY")]}),
("Its graduates also give lectures on the various branches of medicine and Wirtschaftsinformatik - Informationsmanagement und Data Science requisite for the degree of doctor of medicine, and those extra-academical courses are recognized, under certain restrictions, by the University Court, as qualifying for the degree.", {"entities": [(73,136, "STUDY")]}),
("Its graduates also give lectures on the various branches of medicine and Wirtschaftsinformatik - Informationsmanagement und Data Science requisite for the degree of doctor of medicine, and those extra-academical courses are recognized, under certain restrictions, by the University Court, as qualifying for the degree.", {"entities": [(97,136, "STUDY")]}),
("Meanwhile the progress of letters, Wirtschaftsinformatik - Informationsmanagement und Data Science and learning manifested the recovery of the city.", {"entities": [(35,56, "STUDY")]}),
("Meanwhile the progress of letters, Wirtschaftsinformatik - Informationsmanagement und Data Science and learning manifested the recovery of the city.", {"entities": [(35,98, "STUDY")]}),
("Meanwhile the progress of letters, Wirtschaftsinformatik - Informationsmanagement und Data Science and learning manifested the recovery of the city.", {"entities": [(59,98, "STUDY")]}),
("The university provides instruction and grants degrees in arts, law, medicine, Wirtschaftsinformatik - Informationsmanagement und Data Science and engineering; instruction in theology, however, is given, not by the university, but by the different affiliated colleges.", {"entities": [(79,100, "STUDY")]}),
("The university provides instruction and grants degrees in arts, law, medicine, Wirtschaftsinformatik - Informationsmanagement und Data Science and engineering; instruction in theology, however, is given, not by the university, but by the different affiliated colleges.", {"entities": [(79,142, "STUDY")]}),
("The university provides instruction and grants degrees in arts, law, medicine, Wirtschaftsinformatik - Informationsmanagement und Data Science and engineering; instruction in theology, however, is given, not by the university, but by the different affiliated colleges.", {"entities": [(103,142, "STUDY")]}),
("Roger Bacon was rather a pioneer of Wirtschaftsinformatik - Informationsmanagement und Data Science than a Scholastic, and persecution and imprisonment were the penalty of his opposition to the spirit of his time.", {"entities": [(36,57, "STUDY")]}),
("Roger Bacon was rather a pioneer of Wirtschaftsinformatik - Informationsmanagement und Data Science than a Scholastic, and persecution and imprisonment were the penalty of his opposition to the spirit of his time.", {"entities": [(36,99, "STUDY")]}),
("Roger Bacon was rather a pioneer of Wirtschaftsinformatik - Informationsmanagement und Data Science than a Scholastic, and persecution and imprisonment were the penalty of his opposition to the spirit of his time.", {"entities": [(60,99, "STUDY")]}),
("At the university he made rapid progress, especially in jurisprudence, though preferring the study of Wirtschaftsinformatik - Informationsmanagement und Data Science, literature, juridical science and philosophy.", {"entities": [(102,123, "STUDY")]}),
("At the university he made rapid progress, especially in jurisprudence, though preferring the study of Wirtschaftsinformatik - Informationsmanagement und Data Science, literature, juridical science and philosophy.", {"entities": [(102,165, "STUDY")]}),
("At the university he made rapid progress, especially in jurisprudence, though preferring the study of Wirtschaftsinformatik - Informationsmanagement und Data Science, literature, juridical science and philosophy.", {"entities": [(126,165, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik ", {"entities": [(32,53, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik ", {"entities": [(20,41, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik ", {"entities": [(28,49, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik ", {"entities": [(23,44, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik ", {"entities": [(23,44, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik ", {"entities": [(34,55, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik ", {"entities": [(25,46, "STUDY")]}),
("who can help me with Wirtschaftsinformatik ", {"entities": [(21,42, "STUDY")]}),
("i have a question about Wirtschaftsinformatik ", {"entities": [(24,45, "STUDY")]}),
("Which professor does reasearch in Wirtschaftsinformatik ", {"entities": [(34,55, "STUDY")]}),
("Who has the reasearch area Wirtschaftsinformatik ", {"entities": [(27,48, "STUDY")]}),
("Which professor has this research area Wirtschaftsinformatik ", {"entities": [(39,60, "STUDY")]}),
("Who specializes in Wirtschaftsinformatik ", {"entities": [(19,40, "STUDY")]}),
("I have a problem in Wirtschaft und Verkehr", {"entities": [(20,42, "STUDY")]}),
("I am writing a paper about Wirtschaftsinformatik ", {"entities": [(27,48, "STUDY")]}),
("I have a question about Wirtschaftsinformatik ", {"entities": [(24,45, "STUDY")]}),
("I have a thesis about Wirtschaftsinformatik ", {"entities": [(22,43, "STUDY")]}),
("I am writing a thesis about Wirtschaftsinformatik ", {"entities": [(28,49, "STUDY")]}),
("I have a presentation about Wirtschaftsinformatik ", {"entities": [(28,49, "STUDY")]}),
("I need help in Wirtschaftsinformatik ", {"entities": [(15,36, "STUDY")]}),
("I did some research in Wirtschaftsinformatik ", {"entities": [(23,44, "STUDY")]}),
("I want to ask somebody about Wirtschaftsinformatik ", {"entities": [(29,50, "STUDY")]}),
("I don’t understand Wirtschaftsinformatik ", {"entities": [(19,40, "STUDY")]}),
("He is an expert in Wirtschaftsinformatik ", {"entities": [(19,40, "STUDY")]}),
("He is a professor for Wirtschaftsinformatik ", {"entities": [(22,43, "STUDY")]}),
("I have a test in Wirtschaftsinformatik ", {"entities": [(17,38, "STUDY")]}),
("He teaches Wirtschaftsinformatik ", {"entities": [(11,32, "STUDY")]}),
("I need to learn Wirtschaftsinformatik ", {"entities": [(16,37, "STUDY")]}),
("I have some questions about the topic Wirtschaftsinformatik ", {"entities": [(38,59, "STUDY")]}),
("I need to do more research about Wirtschaftsinformatik ", {"entities": [(33,54, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(32,53, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(32,95, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(56,95, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(20,41, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(20,83, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(44,83, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,49, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,91, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(52,91, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,44, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,86, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(47,86, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,44, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,86, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(47,86, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(34,55, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(34,97, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(58,97, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(25,46, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(25,88, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(49,88, "STUDY")]}),
("who can help me with Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(21,42, "STUDY")]}),
("who can help me with Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(21,84, "STUDY")]}),
("who can help me with Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(45,84, "STUDY")]}),
("i have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(24,45, "STUDY")]}),
("i have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(24,87, "STUDY")]}),
("i have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(48,87, "STUDY")]}),
("Which professor does reasearch in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(34,55, "STUDY")]}),
("Which professor does reasearch in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(34,97, "STUDY")]}),
("Which professor does reasearch in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(58,97, "STUDY")]}),
("Who has the reasearch area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(27,48, "STUDY")]}),
("Who has the reasearch area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(27,90, "STUDY")]}),
("Who has the reasearch area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(51,90, "STUDY")]}),
("Which professor has this research area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(39,60, "STUDY")]}),
("Which professor has this research area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(39,102, "STUDY")]}),
("Which professor has this research area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(63,102, "STUDY")]}),
("Who specializes in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,40, "STUDY")]}),
("Who specializes in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,82, "STUDY")]}),
("Who specializes in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(43,82, "STUDY")]}),
("I have a problem in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(20,41, "STUDY")]}),
("I have a problem in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(20,83, "STUDY")]}),
("I have a problem in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(44,83, "STUDY")]}),
("I am writing a paper about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(27,48, "STUDY")]}),
("I am writing a paper about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(27,90, "STUDY")]}),
("I am writing a paper about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(51,90, "STUDY")]}),
("I have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(24,45, "STUDY")]}),
("I have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(24,87, "STUDY")]}),
("I have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(48,87, "STUDY")]}),
("I have a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(22,43, "STUDY")]}),
("I have a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(22,85, "STUDY")]}),
("I have a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(46,85, "STUDY")]}),
("I am writing a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,49, "STUDY")]}),
("I am writing a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,91, "STUDY")]}),
("I am writing a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(52,91, "STUDY")]}),
("I have a presentation about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,49, "STUDY")]}),
("I have a presentation about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,91, "STUDY")]}),
("I have a presentation about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(52,91, "STUDY")]}),
("I need help in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(15,36, "STUDY")]}),
("I need help in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(15,78, "STUDY")]}),
("I need help in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(39,78, "STUDY")]}),
("I did some research in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,44, "STUDY")]}),
("I did some research in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,86, "STUDY")]}),
("I did some research in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(47,86, "STUDY")]}),
("I want to ask somebody about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(29,50, "STUDY")]}),
("I want to ask somebody about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(29,92, "STUDY")]}),
("I want to ask somebody about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(53,92, "STUDY")]}),
("I don’t understand Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,40, "STUDY")]}),
("I don’t understand Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,82, "STUDY")]}),
("I don’t understand Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(43,82, "STUDY")]}),
("He is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,40, "STUDY")]}),
("He is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,82, "STUDY")]}),
("He is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(43,82, "STUDY")]}),
("He is a professor for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(22,43, "STUDY")]}),
("He is a professor for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(22,85, "STUDY")]}),
("He is a professor for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(46,85, "STUDY")]}),
("I have a test in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(17,38, "STUDY")]}),
("I have a test in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(17,80, "STUDY")]}),
("I have a test in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(41,80, "STUDY")]}),
("He teaches Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(11,32, "STUDY")]}),
("He teaches Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(11,74, "STUDY")]}),
("He teaches Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(35,74, "STUDY")]}),
("I need to learn Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(16,37, "STUDY")]}),
("I need to learn Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(16,79, "STUDY")]}),
("I need to learn Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(40,79, "STUDY")]}),
("I have some questions about the topic Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(38,59, "STUDY")]}),
("I have some questions about the topic Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(38,101, "STUDY")]}),
("I have some questions about the topic Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(62,101, "STUDY")]}),
("I need to do more research about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(33,54, "STUDY")]}),
("I need to do more research about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(33,96, "STUDY")]}),
("I need to do more research about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(57,96, "STUDY")]}),
("Which professor is an expert in Digitale Transformation ", {"entities": [(32,55, "STUDY")]}),
("Who is an expert in Digitale Transformation ", {"entities": [(20,43, "STUDY")]}),
("I\'m lookin for an expert in Digitale Transformation ", {"entities": [(28,51, "STUDY")]}),
("Is there an expert for Digitale Transformation ", {"entities": [(23,46, "STUDY")]}),
("i search an expert for Digitale Transformation ", {"entities": [(23,46, "STUDY")]}),
("Which professor knows a lot about Digitale Transformation ", {"entities": [(34,57, "STUDY")]}),
("Who knows the most about Digitale Transformation ", {"entities": [(25,48, "STUDY")]}),
("who can help me with Digitale Transformation ", {"entities": [(21,44, "STUDY")]}),
("i have a question about Digitale Transformation ", {"entities": [(24,47, "STUDY")]}),
("Which professor does reasearch in Digitale Transformation ", {"entities": [(34,57, "STUDY")]}),
("Who has the reasearch area Digitale Transformation ", {"entities": [(27,50, "STUDY")]}),
("Which professor has this research area Digitale Transformation ", {"entities": [(39,62, "STUDY")]}),
("Who specializes in Digitale Transformation ", {"entities": [(19,42, "STUDY")]}),
("I have a problem in Digitale Transformation ", {"entities": [(20,43, "STUDY")]}),
("I am writing a paper about Digitale Transformation ", {"entities": [(27,50, "STUDY")]}),
("I have a question about Digitale Transformation ", {"entities": [(24,47, "STUDY")]}),
("I have a thesis about Digitale Transformation ", {"entities": [(22,45, "STUDY")]}),
("I am writing a thesis about Digitale Transformation ", {"entities": [(28,51, "STUDY")]}),
("I have a presentation about Digitale Transformation ", {"entities": [(28,51, "STUDY")]}),
("I need help in Digitale Transformation ", {"entities": [(15,38, "STUDY")]}),
("I did some research in Digitale Transformation ", {"entities": [(23,46, "STUDY")]}),
("I want to ask somebody about Digitale Transformation ", {"entities": [(29,52, "STUDY")]}),
("I don’t understand Digitale Transformation ", {"entities": [(19,42, "STUDY")]}),
("He is an expert in Digitale Transformation ", {"entities": [(19,42, "STUDY")]}),
("He is a professor for Digitale Transformation ", {"entities": [(22,45, "STUDY")]}),
("I have a test in Digitale Transformation ", {"entities": [(17,40, "STUDY")]}),
("He teaches Digitale Transformation ", {"entities": [(11,34, "STUDY")]}),
("I need to learn Digitale Transformation ", {"entities": [(16,39, "STUDY")]}),
("I have some questions about the topic Digitale Transformation ", {"entities": [(38,61, "STUDY")]}),
("I need to do more research about Digitale Transformation ", {"entities": [(33,56, "STUDY")]}),
("Which professor is an expert in International Business ", {"entities": [(32,54, "STUDY")]}),
("Who is an expert in International Business ", {"entities": [(20,42, "STUDY")]}),
("I\'m lookin for an expert in International Business ", {"entities": [(28,50, "STUDY")]}),
("Is there an expert for International Business ", {"entities": [(23,45, "STUDY")]}),
("i search an expert for International Business ", {"entities": [(23,45, "STUDY")]}),
("Which professor knows a lot about International Business ", {"entities": [(34,56, "STUDY")]}),
("Who knows the most about International Business ", {"entities": [(25,47, "STUDY")]}),
("who can help me with International Business ", {"entities": [(21,43, "STUDY")]}),
("i have a question about International Business ", {"entities": [(24,46, "STUDY")]}),
("Which professor does reasearch in International Business ", {"entities": [(34,56, "STUDY")]}),
("Who has the reasearch area International Business ", {"entities": [(27,49, "STUDY")]}),
("Which professor has this research area International Business ", {"entities": [(39,61, "STUDY")]}),
("Who specializes in International Business ", {"entities": [(19,41, "STUDY")]}),
("I have a problem in International Business ", {"entities": [(20,42, "STUDY")]}),
("I am writing a paper about International Business ", {"entities": [(27,49, "STUDY")]}),
("I have a question about International Business ", {"entities": [(24,46, "STUDY")]}),
("I have a thesis about International Business ", {"entities": [(22,44, "STUDY")]}),
("I am writing a thesis about International Business ", {"entities": [(28,50, "STUDY")]}),
("I have a presentation about International Business ", {"entities": [(28,50, "STUDY")]}),
("I need help in International Business ", {"entities": [(15,37, "STUDY")]}),
("I did some research in International Business ", {"entities": [(23,45, "STUDY")]}),
("I want to ask somebody about International Business ", {"entities": [(29,51, "STUDY")]}),
("I don’t understand International Business ", {"entities": [(19,41, "STUDY")]}),
("He is an expert in International Business ", {"entities": [(19,41, "STUDY")]}),
("He is a professor for International Business ", {"entities": [(22,44, "STUDY")]}),
("I have a test in International Business ", {"entities": [(17,39, "STUDY")]}),
("He teaches International Business ", {"entities": [(11,33, "STUDY")]}),
("I need to learn International Business ", {"entities": [(16,38, "STUDY")]}),
("I have some questions about the topic International Business ", {"entities": [(38,60, "STUDY")]}),
("I need to do more research about International Business ", {"entities": [(33,55, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(32,53, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(32,95, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(56,95, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(20,41, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(20,83, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(44,83, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,49, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,91, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(52,91, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,44, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,86, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(47,86, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,44, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,86, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(47,86, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(34,55, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(34,97, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(58,97, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(25,46, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(25,88, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(49,88, "STUDY")]}),
("who can help me with Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(21,42, "STUDY")]}),
("who can help me with Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(21,84, "STUDY")]}),
("who can help me with Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(45,84, "STUDY")]}),
("i have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(24,45, "STUDY")]}),
("i have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(24,87, "STUDY")]}),
("i have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(48,87, "STUDY")]}),
("Which professor does reasearch in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(34,55, "STUDY")]}),
("Which professor does reasearch in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(34,97, "STUDY")]}),
("Which professor does reasearch in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(58,97, "STUDY")]}),
("Who has the reasearch area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(27,48, "STUDY")]}),
("Who has the reasearch area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(27,90, "STUDY")]}),
("Who has the reasearch area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(51,90, "STUDY")]}),
("Which professor has this research area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(39,60, "STUDY")]}),
("Which professor has this research area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(39,102, "STUDY")]}),
("Which professor has this research area Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(63,102, "STUDY")]}),
("Who specializes in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,40, "STUDY")]}),
("Who specializes in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,82, "STUDY")]}),
("Who specializes in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(43,82, "STUDY")]}),
("I have a problem in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(20,41, "STUDY")]}),
("I have a problem in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(20,83, "STUDY")]}),
("I have a problem in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(44,83, "STUDY")]}),
("I am writing a paper about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(27,48, "STUDY")]}),
("I am writing a paper about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(27,90, "STUDY")]}),
("I am writing a paper about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(51,90, "STUDY")]}),
("I have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(24,45, "STUDY")]}),
("I have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(24,87, "STUDY")]}),
("I have a question about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(48,87, "STUDY")]}),
("I have a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(22,43, "STUDY")]}),
("I have a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(22,85, "STUDY")]}),
("I have a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(46,85, "STUDY")]}),
("I am writing a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,49, "STUDY")]}),
("I am writing a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,91, "STUDY")]}),
("I am writing a thesis about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(52,91, "STUDY")]}),
("I have a presentation about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,49, "STUDY")]}),
("I have a presentation about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(28,91, "STUDY")]}),
("I have a presentation about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(52,91, "STUDY")]}),
("I need help in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(15,36, "STUDY")]}),
("I need help in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(15,78, "STUDY")]}),
("I need help in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(39,78, "STUDY")]}),
("I did some research in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,44, "STUDY")]}),
("I did some research in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(23,86, "STUDY")]}),
("I did some research in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(47,86, "STUDY")]}),
("I want to ask somebody about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(29,50, "STUDY")]}),
("I want to ask somebody about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(29,92, "STUDY")]}),
("I want to ask somebody about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(53,92, "STUDY")]}),
("I don’t understand Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,40, "STUDY")]}),
("I don’t understand Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,82, "STUDY")]}),
("I don’t understand Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(43,82, "STUDY")]}),
("He is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,40, "STUDY")]}),
("He is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(19,82, "STUDY")]}),
("He is an expert in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(43,82, "STUDY")]}),
("He is a professor for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(22,43, "STUDY")]}),
("He is a professor for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(22,85, "STUDY")]}),
("He is a professor for Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(46,85, "STUDY")]}),
("I have a test in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(17,38, "STUDY")]}),
("I have a test in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(17,80, "STUDY")]}),
("I have a test in Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(41,80, "STUDY")]}),
("He teaches Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(11,32, "STUDY")]}),
("He teaches Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(11,74, "STUDY")]}),
("He teaches Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(35,74, "STUDY")]}),
("I need to learn Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(16,37, "STUDY")]}),
("I need to learn Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(16,79, "STUDY")]}),
("I need to learn Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(40,79, "STUDY")]}),
("I have some questions about the topic Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(38,59, "STUDY")]}),
("I have some questions about the topic Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(38,101, "STUDY")]}),
("I have some questions about the topic Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(62,101, "STUDY")]}),
("I need to do more research about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(33,54, "STUDY")]}),
("I need to do more research about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(33,96, "STUDY")]}),
("I need to do more research about Wirtschaftsinformatik - Informationsmanagement und Data Science ", {"entities": [(57,96, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik ", {"entities": [(32,53, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik ", {"entities": [(20,41, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik ", {"entities": [(28,49, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik ", {"entities": [(23,44, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik ", {"entities": [(23,44, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik ", {"entities": [(34,55, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik ", {"entities": [(25,46, "STUDY")]}),
("who can help me with Wirtschaftsinformatik ", {"entities": [(21,42, "STUDY")]}),
("i have a question about Wirtschaftsinformatik ", {"entities": [(24,45, "STUDY")]}),
("Which professor does reasearch in Wirtschaftsinformatik ", {"entities": [(34,55, "STUDY")]}),
("Who has the reasearch area Wirtschaftsinformatik ", {"entities": [(27,48, "STUDY")]}),
("Which professor has this research area Wirtschaftsinformatik ", {"entities": [(39,60, "STUDY")]}),
("Who specializes in Wirtschaftsinformatik ", {"entities": [(19,40, "STUDY")]}),
("I have a problem in Wirtschaftsinformatik ", {"entities": [(20,41, "STUDY")]}),
("I am writing a paper about Wirtschaftsinformatik ", {"entities": [(27,48, "STUDY")]}),
("I have a question about Wirtschaftsinformatik ", {"entities": [(24,45, "STUDY")]}),
("I have a thesis about Wirtschaftsinformatik ", {"entities": [(22,43, "STUDY")]}),
("I am writing a thesis about Wirtschaftsinformatik ", {"entities": [(28,49, "STUDY")]}),
("I have a presentation about Wirtschaftsinformatik ", {"entities": [(28,49, "STUDY")]}),
("I need help in Wirtschaftsinformatik ", {"entities": [(15,36, "STUDY")]}),
("I did some research in Wirtschaftsinformatik ", {"entities": [(23,44, "STUDY")]}),
("I want to ask somebody about Wirtschaftsinformatik ", {"entities": [(29,50, "STUDY")]}),
("I don’t understand Wirtschaftsinformatik ", {"entities": [(19,40, "STUDY")]}),
("He is an expert in Wirtschaftsinformatik ", {"entities": [(19,40, "STUDY")]}),
("He is a professor for Wirtschaftsinformatik ", {"entities": [(22,43, "STUDY")]}),
("I have a test in Wirtschaftsinformatik ", {"entities": [(17,38, "STUDY")]}),
("He teaches Wirtschaftsinformatik ", {"entities": [(11,32, "STUDY")]}),
("I need to learn Wirtschaftsinformatik ", {"entities": [(16,37, "STUDY")]}),
("I have some questions about the topic Wirtschaftsinformatik ", {"entities": [(38,59, "STUDY")]}),
("I need to do more research about Wirtschaftsinformatik ", {"entities": [(33,54, "STUDY")]}),
("Which professor is an expert in Software Engineering ", {"entities": [(32,52, "STUDY")]}),
("Who is an expert in Software Engineering ", {"entities": [(20,40, "STUDY")]}),
("I\'m lookin for an expert in Software Engineering ", {"entities": [(28,48, "STUDY")]}),
("Is there an expert for Software Engineering ", {"entities": [(23,43, "STUDY")]}),
("i search an expert for Software Engineering ", {"entities": [(23,43, "STUDY")]}),
("Which professor knows a lot about Software Engineering ", {"entities": [(34,54, "STUDY")]}),
("Who knows the most about Software Engineering ", {"entities": [(25,45, "STUDY")]}),
("who can help me with Software Engineering ", {"entities": [(21,41, "STUDY")]}),
("i have a question about Software Engineering ", {"entities": [(24,44, "STUDY")]}),
("Which professor does reasearch in Software Engineering ", {"entities": [(34,54, "STUDY")]}),
("Who has the reasearch area Software Engineering ", {"entities": [(27,47, "STUDY")]}),
("Which professor has this research area Software Engineering ", {"entities": [(39,59, "STUDY")]}),
("Who specializes in Software Engineering ", {"entities": [(19,39, "STUDY")]}),
("I have a problem in Software Engineering ", {"entities": [(20,40, "STUDY")]}),
("I am writing a paper about Software Engineering ", {"entities": [(27,47, "STUDY")]}),
("I have a question about Software Engineering ", {"entities": [(24,44, "STUDY")]}),
("I have a thesis about Software Engineering ", {"entities": [(22,42, "STUDY")]}),
("I am writing a thesis about Software Engineering ", {"entities": [(28,48, "STUDY")]}),
("I have a presentation about Software Engineering ", {"entities": [(28,48, "STUDY")]}),
("I need help in Software Engineering ", {"entities": [(15,35, "STUDY")]}),
("I did some research in Software Engineering ", {"entities": [(23,43, "STUDY")]}),
("I want to ask somebody about Software Engineering ", {"entities": [(29,49, "STUDY")]}),
("I don’t understand Software Engineering ", {"entities": [(19,39, "STUDY")]}),
("He is an expert in Software Engineering ", {"entities": [(19,39, "STUDY")]}),
("He is a professor for Software Engineering ", {"entities": [(22,42, "STUDY")]}),
("I have a test in Software Engineering ", {"entities": [(17,37, "STUDY")]}),
("He teaches Software Engineering ", {"entities": [(11,31, "STUDY")]}),
("I need to learn Software Engineering ", {"entities": [(16,36, "STUDY")]}),
("I have some questions about the topic Software Engineering ", {"entities": [(38,58, "STUDY")]}),
("I need to do more research about Software Engineering ", {"entities": [(33,53, "STUDY")]}),
("Which professor is an expert in Software Engineering ", {"entities": [(32,52, "STUDY")]}),
("Who is an expert in Software Engineering ", {"entities": [(20,40, "STUDY")]}),
("Which professor is an expert in Software Engineering ", {"entities": [(32,52, "STUDY")]}),
("Who is an expert in Software Engineering ", {"entities": [(20,40, "STUDY")]}),
("I\'m lookin for an expert in Software Engineering ", {"entities": [(28,48, "STUDY")]}),
("Is there an expert for Software Engineering ", {"entities": [(23,43, "STUDY")]}),
("i search an expert for Software Engineering ", {"entities": [(23,43, "STUDY")]}),
("Which professor knows a lot about Software Engineering ", {"entities": [(34,54, "STUDY")]}),
("Who knows the most about Software Engineering ", {"entities": [(25,45, "STUDY")]}),
("who can help me with Software Engineering ", {"entities": [(21,41, "STUDY")]}),
("i have a question about Software Engineering ", {"entities": [(24,44, "STUDY")]}),
("Which professor does reasearch in Software Engineering ", {"entities": [(34,54, "STUDY")]}),
("Who has the reasearch area Software Engineering ", {"entities": [(27,47, "STUDY")]}),
("Which professor has this research area Software Engineering ", {"entities": [(39,59, "STUDY")]}),
("Who specializes in Software Engineering ", {"entities": [(19,39, "STUDY")]}),
("I have a problem in Software Engineering ", {"entities": [(20,40, "STUDY")]}),
("I am writing a paper about Software Engineering ", {"entities": [(27,47, "STUDY")]}),
("I have a question about Software Engineering ", {"entities": [(24,44, "STUDY")]}),
("I have a thesis about Software Engineering ", {"entities": [(22,42, "STUDY")]}),
("I am writing a thesis about Software Engineering ", {"entities": [(28,48, "STUDY")]}),
("I have a presentation about Software Engineering ", {"entities": [(28,48, "STUDY")]}),
("I need help in Software Engineering ", {"entities": [(15,35, "STUDY")]}),
("I did some research in Software Engineering ", {"entities": [(23,43, "STUDY")]}),
("I want to ask somebody about Software Engineering ", {"entities": [(29,49, "STUDY")]}),
("I don’t understand Software Engineering ", {"entities": [(19,39, "STUDY")]}),
("He is an expert in Software Engineering ", {"entities": [(19,39, "STUDY")]}),
("He is a professor for Software Engineering ", {"entities": [(22,42, "STUDY")]}),
("I have a test in Software Engineering ", {"entities": [(17,37, "STUDY")]}),
("He teaches Software Engineering ", {"entities": [(11,31, "STUDY")]}),
("I need to learn Software Engineering ", {"entities": [(16,36, "STUDY")]}),
("I have some questions about the topic Software Engineering ", {"entities": [(38,58, "STUDY")]}),
("I need to do more research about Software Engineering ", {"entities": [(33,53, "STUDY")]}),
("Which professor is an expert in Software Engineering ", {"entities": [(32,52, "STUDY")]}),
("Who is an expert in Software Engineering ", {"entities": [(20,40, "STUDY")]}),
("Which professor is an expert in Software Engineering ", {"entities": [(32,52, "STUDY")]}),
("Who is an expert in Software Engineering ", {"entities": [(20,40, "STUDY")]}),
("I\'m lookin for an expert in Software Engineering ", {"entities": [(28,48, "STUDY")]}),
("Is there an expert for Software Engineering ", {"entities": [(23,43, "STUDY")]}),
("i search an expert for Software Engineering ", {"entities": [(23,43, "STUDY")]}),
("Which professor knows a lot about Software Engineering ", {"entities": [(34,54, "STUDY")]}),
("Who knows the most about Software Engineering ", {"entities": [(25,45, "STUDY")]}),
("who can help me with Software Engineering ", {"entities": [(21,41, "STUDY")]}),
("i have a question about Software Engineering ", {"entities": [(24,44, "STUDY")]}),
("Which professor does reasearch in Software Engineering ", {"entities": [(34,54, "STUDY")]}),
("Who has the reasearch area Software Engineering ", {"entities": [(27,47, "STUDY")]}),
("Which professor has this research area Software Engineering ", {"entities": [(39,59, "STUDY")]}),
("Who specializes in Software Engineering ", {"entities": [(19,39, "STUDY")]}),
("I have a problem in Software Engineering ", {"entities": [(20,40, "STUDY")]}),
("I am writing a paper about Software Engineering ", {"entities": [(27,47, "STUDY")]}),
("I have a question about Software Engineering ", {"entities": [(24,44, "STUDY")]}),
("I have a thesis about Software Engineering ", {"entities": [(22,42, "STUDY")]}),
("I am writing a thesis about Software Engineering ", {"entities": [(28,48, "STUDY")]}),
("I have a presentation about Software Engineering ", {"entities": [(28,48, "STUDY")]}),
("I need help in Software Engineering ", {"entities": [(15,35, "STUDY")]}),
("I did some research in Software Engineering ", {"entities": [(23,43, "STUDY")]}),
("I want to ask somebody about Software Engineering ", {"entities": [(29,49, "STUDY")]}),
("I don’t understand Software Engineering ", {"entities": [(19,39, "STUDY")]}),
("He is an expert in Software Engineering ", {"entities": [(19,39, "STUDY")]}),
("He is a professor for Software Engineering ", {"entities": [(22,42, "STUDY")]}),
("I have a test in Software Engineering ", {"entities": [(17,37, "STUDY")]}),
("He teaches Software Engineering ", {"entities": [(11,31, "STUDY")]}),
("I need to learn Software Engineering ", {"entities": [(16,36, "STUDY")]}),
("I have some questions about the topic Software Engineering ", {"entities": [(38,58, "STUDY")]}),
("I need to do more research about Software Engineering ", {"entities": [(33,53, "STUDY")]}),
("Which professor is an expert in Software Engineering ", {"entities": [(32,52, "STUDY")]}),
("Who is an expert in Software Engineering ", {"entities": [(20,40, "STUDY")]}),
("Which professor is an expert in Management und Vertrieb ", {"entities": [(32,55, "STUDY")]}),
("Who is an expert in Management und Vertrieb ", {"entities": [(20,43, "STUDY")]}),
("I\'m lookin for an expert in Management und Vertrieb ", {"entities": [(28,51, "STUDY")]}),
("Is there an expert for Management und Vertrieb ", {"entities": [(23,46, "STUDY")]}),
("i search an expert for Management und Vertrieb ", {"entities": [(23,46, "STUDY")]}),
("Which professor knows a lot about Management und Vertrieb ", {"entities": [(34,57, "STUDY")]}),
("Who knows the most about Management und Vertrieb ", {"entities": [(25,48, "STUDY")]}),
("who can help me with Management und Vertrieb ", {"entities": [(21,44, "STUDY")]}),
("i have a question about Management und Vertrieb ", {"entities": [(24,47, "STUDY")]}),
("Which professor does reasearch in Management und Vertrieb ", {"entities": [(34,57, "STUDY")]}),
("Who has the reasearch area Management und Vertrieb ", {"entities": [(27,50, "STUDY")]}),
("Which professor has this research area Management und Vertrieb ", {"entities": [(39,62, "STUDY")]}),
("Who specializes in Management und Vertrieb ", {"entities": [(19,42, "STUDY")]}),
("I have a problem in Management und Vertrieb ", {"entities": [(20,43, "STUDY")]}),
("I am writing a paper about Management und Vertrieb ", {"entities": [(27,50, "STUDY")]}),
("I have a question about Management und Vertrieb ", {"entities": [(24,47, "STUDY")]}),
("I have a thesis about Management und Vertrieb ", {"entities": [(22,45, "STUDY")]}),
("I am writing a thesis about Management und Vertrieb ", {"entities": [(28,51, "STUDY")]}),
("I have a presentation about Management und Vertrieb ", {"entities": [(28,51, "STUDY")]}),
("I need help in Management und Vertrieb ", {"entities": [(15,38, "STUDY")]}),
("I did some research in Management und Vertrieb ", {"entities": [(23,46, "STUDY")]}),
("I want to ask somebody about Management und Vertrieb ", {"entities": [(29,52, "STUDY")]}),
("I don’t understand Management und Vertrieb ", {"entities": [(19,42, "STUDY")]}),
("He is an expert in Management und Vertrieb ", {"entities": [(19,42, "STUDY")]}),
("He is a professor for Management und Vertrieb ", {"entities": [(22,45, "STUDY")]}),
("I have a test in Management und Vertrieb ", {"entities": [(17,40, "STUDY")]}),
("He teaches Management und Vertrieb ", {"entities": [(11,34, "STUDY")]}),
("I need to learn Management und Vertrieb ", {"entities": [(16,39, "STUDY")]}),
("I have some questions about the topic Management und Vertrieb ", {"entities": [(38,61, "STUDY")]}),
("I need to do more research about Management und Vertrieb ", {"entities": [(33,56, "STUDY")]}),
("Which professor is an expert in Management und Vertrieb ", {"entities": [(32,55, "STUDY")]}),
("Who is an expert in Management und Vertrieb ", {"entities": [(20,43, "STUDY")]}),
("Which professor is an expert in International Tourism Management ", {"entities": [(32,64, "STUDY")]}),
("Who is an expert in International Tourism Management ", {"entities": [(20,52, "STUDY")]}),
("I\'m lookin for an expert in International Tourism Management ", {"entities": [(28,60, "STUDY")]}),
("Is there an expert for International Tourism Management ", {"entities": [(23,55, "STUDY")]}),
("i search an expert for International Tourism Management ", {"entities": [(23,55, "STUDY")]}),
("Which professor knows a lot about International Tourism Management ", {"entities": [(34,66, "STUDY")]}),
("Who knows the most about International Tourism Management ", {"entities": [(25,57, "STUDY")]}),
("who can help me with International Tourism Management ", {"entities": [(21,53, "STUDY")]}),
("i have a question about International Tourism Management ", {"entities": [(24,56, "STUDY")]}),
("Which professor does reasearch in International Tourism Management ", {"entities": [(34,66, "STUDY")]}),
("Who has the reasearch area International Tourism Management ", {"entities": [(27,59, "STUDY")]}),
("Which professor has this research area International Tourism Management ", {"entities": [(39,71, "STUDY")]}),
("Who specializes in International Tourism Management ", {"entities": [(19,51, "STUDY")]}),
("I have a problem in International Tourism Management ", {"entities": [(20,52, "STUDY")]}),
("I am writing a paper about International Tourism Management ", {"entities": [(27,59, "STUDY")]}),
("I have a question about International Tourism Management ", {"entities": [(24,56, "STUDY")]}),
("I have a thesis about International Tourism Management ", {"entities": [(22,54, "STUDY")]}),
("I am writing a thesis about International Tourism Management ", {"entities": [(28,60, "STUDY")]}),
("I have a presentation about International Tourism Management ", {"entities": [(28,60, "STUDY")]}),
("I need help in International Tourism Management ", {"entities": [(15,47, "STUDY")]}),
("I did some research in International Tourism Management ", {"entities": [(23,55, "STUDY")]}),
("I want to ask somebody about International Tourism Management ", {"entities": [(29,61, "STUDY")]}),
("I don’t understand International Tourism Management ", {"entities": [(19,51, "STUDY")]}),
("He is an expert in International Tourism Management ", {"entities": [(19,51, "STUDY")]}),
("He is a professor for International Tourism Management ", {"entities": [(22,54, "STUDY")]}),
("I have a test in International Tourism Management ", {"entities": [(17,49, "STUDY")]}),
("He teaches International Tourism Management ", {"entities": [(11,43, "STUDY")]}),
("I need to learn International Tourism Management ", {"entities": [(16,48, "STUDY")]}),
("I have some questions about the topic International Tourism Management ", {"entities": [(38,70, "STUDY")]}),
("I need to do more research about International Tourism Management ", {"entities": [(33,65, "STUDY")]}),
("Which professor is an expert in International Tourism Management ", {"entities": [(32,64, "STUDY")]}),
("Who is an expert in International Tourism Management ", {"entities": [(20,52, "STUDY")]}),
("Which professor is an expert in Intercultural Management ", {"entities": [(32,56, "STUDY")]}),
("Who is an expert in Intercultural Management ", {"entities": [(20,44, "STUDY")]}),
("I\'m lookin for an expert in Intercultural Management ", {"entities": [(28,52, "STUDY")]}),
("Is there an expert for Intercultural Management ", {"entities": [(23,47, "STUDY")]}),
("i search an expert for Intercultural Management ", {"entities": [(23,47, "STUDY")]}),
("Which professor knows a lot about Intercultural Management ", {"entities": [(34,58, "STUDY")]}),
("Who knows the most about Intercultural Management ", {"entities": [(25,49, "STUDY")]}),
("who can help me with Intercultural Management ", {"entities": [(21,45, "STUDY")]}),
("i have a question about Intercultural Management ", {"entities": [(24,48, "STUDY")]}),
("Which professor does reasearch in Intercultural Management ", {"entities": [(34,58, "STUDY")]}),
("Who has the reasearch area Intercultural Management ", {"entities": [(27,51, "STUDY")]}),
("Which professor has this research area Intercultural Management ", {"entities": [(39,63, "STUDY")]}),
("Who specializes in Intercultural Management ", {"entities": [(19,43, "STUDY")]}),
("I have a problem in Intercultural Management ", {"entities": [(20,44, "STUDY")]}),
("I am writing a paper about Intercultural Management ", {"entities": [(27,51, "STUDY")]}),
("I have a question about Intercultural Management ", {"entities": [(24,48, "STUDY")]}),
("I have a thesis about Intercultural Management ", {"entities": [(22,46, "STUDY")]}),
("I am writing a thesis about Intercultural Management ", {"entities": [(28,52, "STUDY")]}),
("I have a presentation about Intercultural Management ", {"entities": [(28,52, "STUDY")]}),
("I need help in Intercultural Management ", {"entities": [(15,39, "STUDY")]}),
("I did some research in Intercultural Management ", {"entities": [(23,47, "STUDY")]}),
("I want to ask somebody about Intercultural Management ", {"entities": [(29,53, "STUDY")]}),
("I don’t understand Intercultural Management ", {"entities": [(19,43, "STUDY")]}),
("He is an expert in Intercultural Management ", {"entities": [(19,43, "STUDY")]}),
("He is a professor for Intercultural Management ", {"entities": [(22,46, "STUDY")]}),
("I have a test in Intercultural Management ", {"entities": [(17,41, "STUDY")]}),
("He teaches Intercultural Management ", {"entities": [(11,35, "STUDY")]}),
("I need to learn Intercultural Management ", {"entities": [(16,40, "STUDY")]}),
("I have some questions about the topic Intercultural Management ", {"entities": [(38,62, "STUDY")]}),
("I need to do more research about Intercultural Management ", {"entities": [(33,57, "STUDY")]}),
("Which professor is an expert in Intercultural Management ", {"entities": [(32,56, "STUDY")]}),
("Who is an expert in Intercultural Management ", {"entities": [(20,44, "STUDY")]}),
("Which professor is an expert in Weinmarketing und Management ", {"entities": [(32,60, "STUDY")]}),
("Who is an expert in Weinmarketing und Management ", {"entities": [(20,48, "STUDY")]}),
("I\'m lookin for an expert in Weinmarketing und Management ", {"entities": [(28,56, "STUDY")]}),
("Is there an expert for Weinmarketing und Management ", {"entities": [(23,51, "STUDY")]}),
("i search an expert for Weinmarketing und Management ", {"entities": [(23,51, "STUDY")]}),
("Which professor knows a lot about Weinmarketing und Management ", {"entities": [(34,62, "STUDY")]}),
("Who knows the most about Weinmarketing und Management ", {"entities": [(25,53, "STUDY")]}),
("who can help me with Weinmarketing und Management ", {"entities": [(21,49, "STUDY")]}),
("i have a question about Weinmarketing und Management ", {"entities": [(24,52, "STUDY")]}),
("Which professor does reasearch in Weinmarketing und Management ", {"entities": [(34,62, "STUDY")]}),
("Who has the reasearch area Weinmarketing und Management ", {"entities": [(27,55, "STUDY")]}),
("Which professor has this research area Weinmarketing und Management ", {"entities": [(39,67, "STUDY")]}),
("Who specializes in Weinmarketing und Management ", {"entities": [(19,47, "STUDY")]}),
("I have a problem in Weinmarketing und Management ", {"entities": [(20,48, "STUDY")]}),
("I am writing a paper about Weinmarketing und Management ", {"entities": [(27,55, "STUDY")]}),
("I have a question about Weinmarketing und Management ", {"entities": [(24,52, "STUDY")]}),
("I have a thesis about Weinmarketing und Management ", {"entities": [(22,50, "STUDY")]}),
("I am writing a thesis about Weinmarketing und Management ", {"entities": [(28,56, "STUDY")]}),
("I have a presentation about Weinmarketing und Management ", {"entities": [(28,56, "STUDY")]}),
("I need help in Weinmarketing und Management ", {"entities": [(15,43, "STUDY")]}),
("I did some research in Weinmarketing und Management ", {"entities": [(23,51, "STUDY")]}),
("I want to ask somebody about Weinmarketing und Management ", {"entities": [(29,57, "STUDY")]}),
("I don’t understand Weinmarketing und Management ", {"entities": [(19,47, "STUDY")]}),
("He is an expert in Weinmarketing und Management ", {"entities": [(19,47, "STUDY")]}),
("He is a professor for Weinmarketing und Management ", {"entities": [(22,50, "STUDY")]}),
("I have a test in Weinmarketing und Management ", {"entities": [(17,45, "STUDY")]}),
("He teaches Weinmarketing und Management ", {"entities": [(11,39, "STUDY")]}),
("I need to learn Weinmarketing und Management ", {"entities": [(16,44, "STUDY")]}),
("I have some questions about the topic Weinmarketing und Management ", {"entities": [(38,66, "STUDY")]}),
("I need to do more research about Weinmarketing und Management ", {"entities": [(33,61, "STUDY")]}),
("Which professor is an expert in Weinmarketing und Management ", {"entities": [(32,60, "STUDY")]}),
("Who is an expert in Weinmarketing und Management ", {"entities": [(20,48, "STUDY")]}),
("Which professor is an expert in Betriebswirtschaft ", {"entities": [(32,50, "STUDY")]}),
("Who is an expert in Betriebswirtschaft ", {"entities": [(20,38, "STUDY")]}),
("I\'m lookin for an expert in Betriebswirtschaft ", {"entities": [(28,46, "STUDY")]}),
("Is there an expert for Betriebswirtschaft ", {"entities": [(23,41, "STUDY")]}),
("i search an expert for Betriebswirtschaft ", {"entities": [(23,41, "STUDY")]}),
("Which professor knows a lot about Betriebswirtschaft ", {"entities": [(34,52, "STUDY")]}),
("Who knows the most about Betriebswirtschaft ", {"entities": [(25,43, "STUDY")]}),
("who can help me with Betriebswirtschaft ", {"entities": [(21,39, "STUDY")]}),
("i have a question about Betriebswirtschaft ", {"entities": [(24,42, "STUDY")]}),
("Which professor does reasearch in Betriebswirtschaft ", {"entities": [(34,52, "STUDY")]}),
("Who has the reasearch area Betriebswirtschaft ", {"entities": [(27,45, "STUDY")]}),
("Which professor has this research area Betriebswirtschaft ", {"entities": [(39,57, "STUDY")]}),
("Who specializes in Betriebswirtschaft ", {"entities": [(19,37, "STUDY")]}),
("I have a problem in Betriebswirtschaft ", {"entities": [(20,38, "STUDY")]}),
("I am writing a paper about Betriebswirtschaft ", {"entities": [(27,45, "STUDY")]}),
("I have a question about Betriebswirtschaft ", {"entities": [(24,42, "STUDY")]}),
("I have a thesis about Betriebswirtschaft ", {"entities": [(22,40, "STUDY")]}),
("I am writing a thesis about Betriebswirtschaft ", {"entities": [(28,46, "STUDY")]}),
("I have a presentation about Betriebswirtschaft ", {"entities": [(28,46, "STUDY")]}),
("I need help in Betriebswirtschaft ", {"entities": [(15,33, "STUDY")]}),
("I did some research in Betriebswirtschaft ", {"entities": [(23,41, "STUDY")]}),
("I want to ask somebody about Betriebswirtschaft ", {"entities": [(29,47, "STUDY")]}),
("I don’t understand Betriebswirtschaft ", {"entities": [(19,37, "STUDY")]}),
("He is an expert in Betriebswirtschaft ", {"entities": [(19,37, "STUDY")]}),
("He is a professor for Betriebswirtschaft ", {"entities": [(22,40, "STUDY")]}),
("I have a test in Betriebswirtschaft ", {"entities": [(17,35, "STUDY")]}),
("He teaches Betriebswirtschaft ", {"entities": [(11,29, "STUDY")]}),
("I need to learn Betriebswirtschaft ", {"entities": [(16,34, "STUDY")]}),
("I have some questions about the topic Betriebswirtschaft ", {"entities": [(38,56, "STUDY")]}),
("I need to do more research about Betriebswirtschaft ", {"entities": [(33,51, "STUDY")]}),
("Which professor is an expert in Betriebswirtschaft ", {"entities": [(32,50, "STUDY")]}),
("Who is an expert in Betriebswirtschaft ", {"entities": [(20,38, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(32,53, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(32,97, "STUDY")]}),
("Which professor is an expert in Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(56,79, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(20,41, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(20,85, "STUDY")]}),
("Who is an expert in Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(44,67, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(28,49, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(28,93, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(52,75, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(23,44, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(23,88, "STUDY")]}),
("Is there an expert for Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(47,70, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(23,44, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(23,88, "STUDY")]}),
("i search an expert for Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(47,70, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(34,55, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(34,99, "STUDY")]}),
("Which professor knows a lot about Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(58,81, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(25,46, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(25,90, "STUDY")]}),
("Who knows the most about Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(49,72, "STUDY")]}),
("who can help me with Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(21,42, "STUDY")]}),
("who can help me with Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(21,86, "STUDY")]}),
("who can help me with Wirtschaftsinformatik - Digitale Transformation, berufsbegleitend ", {"entities": [(45,68, "STUDY")]}),
("i have a question about Informationsmanagement und Data Science", {"entities": [(24,63, "STUDY")]}),
("Which professor does reasearch in Informationsmanagement und Data Science", {"entities": [(34,73, "STUDY")]}),
("Who has the reasearch area Informationsmanagement und Data Science", {"entities": [(27,66, "STUDY")]}),
("Which professor has this research area Informationsmanagement und Data Science", {"entities": [(39,78, "STUDY")]}),
("Who specializes in Informationsmanagement und Data Science", {"entities": [(19,58, "STUDY")]}),
("I have a problem in Informationsmanagement und Data Science", {"entities": [(20,59, "STUDY")]}),
("I am writing a paper about Informationsmanagement und Data Science", {"entities": [(27,66, "STUDY")]}),
("I have a question about Informationsmanagement und Data Science", {"entities": [(24,63, "STUDY")]}),
("I have a thesis about Informationsmanagement und Data Science", {"entities": [(22,61, "STUDY")]}),
("I am writing a thesis about Informationsmanagement und Data Science", {"entities": [(28,67, "STUDY")]}),
("I have a presentation about Informationsmanagement und Data Science", {"entities": [(28,67, "STUDY")]}),
("I need help in Informationsmanagement und Data Science", {"entities": [(15,54, "STUDY")]}),
("I did some research in Informationsmanagement und Data Science", {"entities": [(23,62, "STUDY")]}),
("I want to ask somebody about Informationsmanagement und Data Science", {"entities": [(29,68, "STUDY")]}),
("I don’t understand Informationsmanagement und Data Science", {"entities": [(19,58, "STUDY")]}),
("He is an expert in Informationsmanagement und Data Science", {"entities": [(19,58, "STUDY")]}),
("He is a professor for Informationsmanagement und Data Science", {"entities": [(22,61, "STUDY")]}),
("I have a test in Informationsmanagement und Data Science", {"entities": [(17,56, "STUDY")]}),
("He teaches Informationsmanagement und Data Science", {"entities": [(11,50, "STUDY")]}),
("I need to learn Informationsmanagement und Data Science", {"entities": [(16,55, "STUDY")]}),
("I have some questions about the topic Informationsmanagement und Data Science", {"entities": [(38,77, "STUDY")]}),
("I need to do more research about Informationsmanagement und Data Science", {"entities": [(33,72, "STUDY")]}),
("Which professor is an expert in Informationsmanagement und Data Science", {"entities": [(32,71, "STUDY")]}),
("Who is an expert in Informationsmanagement und Data Science", {"entities": [(20,59, "STUDY")]}),
("Which professor is an expert in International Business ", {"entities": [(32,54, "STUDY")]}),
("Who is an expert in International Business ", {"entities": [(20,42, "STUDY")]}),
("I\'m lookin for an expert in International Business ", {"entities": [(28,50, "STUDY")]}),
("Is there an expert for International Business ", {"entities": [(23,45, "STUDY")]}),
("i search an expert for International Business ", {"entities": [(23,45, "STUDY")]}),
("Which professor knows a lot about International Business ", {"entities": [(34,56, "STUDY")]}),
("Who knows the most about International Business ", {"entities": [(25,47, "STUDY")]}),
("who can help me with International Business ", {"entities": [(21,43, "STUDY")]}),
("i have a question about International Business ", {"entities": [(24,46, "STUDY")]}),
("Which professor does reasearch in International Business ", {"entities": [(34,56, "STUDY")]}),
("Who has the reasearch area International Business ", {"entities": [(27,49, "STUDY")]}),
("Which professor has this research area International Business ", {"entities": [(39,61, "STUDY")]}),
("Who specializes in International Business ", {"entities": [(19,41, "STUDY")]}),
("I have a problem in International Business ", {"entities": [(20,42, "STUDY")]}),
("I am writing a paper about International Business ", {"entities": [(27,49, "STUDY")]}),
("I have a question about International Business ", {"entities": [(24,46, "STUDY")]}),
("I have a thesis about International Business ", {"entities": [(22,44, "STUDY")]}),
("I am writing a thesis about International Business ", {"entities": [(28,50, "STUDY")]}),
("I have a presentation about International Business ", {"entities": [(28,50, "STUDY")]}),
("I need help in International Business ", {"entities": [(15,37, "STUDY")]}),
("I did some research in International Business ", {"entities": [(23,45, "STUDY")]}),
("I want to ask somebody about International Business ", {"entities": [(29,51, "STUDY")]}),
("I don’t understand International Business ", {"entities": [(19,41, "STUDY")]}),
("He is an expert in International Business ", {"entities": [(19,41, "STUDY")]}),
("He is a professor for International Business ", {"entities": [(22,44, "STUDY")]}),
("I have a test in International Business ", {"entities": [(17,39, "STUDY")]}),
("He teaches International Business ", {"entities": [(11,33, "STUDY")]}),
("I need to learn International Business ", {"entities": [(16,38, "STUDY")]}),
("I have some questions about the topic International Business ", {"entities": [(38,60, "STUDY")]}),
("I need to do more research about International Business ", {"entities": [(33,55, "STUDY")]}),
("Which professor is an expert in International Business ", {"entities": [(32,54, "STUDY")]}),
("Who is an expert in International Business ", {"entities": [(20,42, "STUDY")]}),
("Which professor is an expert in Wirtschaft und Verkehr ", {"entities": [(32,54, "STUDY")]}),
("Who is an expert in Wirtschaft und Verkehr ", {"entities": [(20,42, "STUDY")]}),
("I\'m lookin for an expert in Wirtschaft und Verkehr ", {"entities": [(28,50, "STUDY")]}),
("Is there an expert for Wirtschaft und Verkehr ", {"entities": [(23,45, "STUDY")]}),
("i search an expert for Wirtschaft und Verkehr ", {"entities": [(23,45, "STUDY")]}),
("Which professor knows a lot about Wirtschaft und Verkehr ", {"entities": [(34,56, "STUDY")]}),
("Who knows the most about Wirtschaft und Verkehr ", {"entities": [(25,47, "STUDY")]}),
("who can help me with Wirtschaft und Verkehr ", {"entities": [(21,43, "STUDY")]}),
("i have a question about Wirtschaft und Verkehr ", {"entities": [(24,46, "STUDY")]}),
("Which professor does reasearch in Wirtschaft und Verkehr ", {"entities": [(34,56, "STUDY")]}),
("Who has the reasearch area Wirtschaft und Verkehr ", {"entities": [(27,49, "STUDY")]}),
("Which professor has this research area Wirtschaft und Verkehr ", {"entities": [(39,61, "STUDY")]}),
("Who specializes in Wirtschaft und Verkehr ", {"entities": [(19,41, "STUDY")]}),
("I have a problem in Wirtschaft und Verkehr ", {"entities": [(20,42, "STUDY")]}),
("I am writing a paper about Wirtschaft und Verkehr ", {"entities": [(27,49, "STUDY")]}),
("I have a question about Wirtschaft und Verkehr ", {"entities": [(24,46, "STUDY")]}),
("I have a thesis about Wirtschaft und Verkehr ", {"entities": [(22,44, "STUDY")]}),
("I am writing a thesis about Wirtschaft und Verkehr ", {"entities": [(28,50, "STUDY")]}),
("I have a presentation about Wirtschaft und Verkehr ", {"entities": [(28,50, "STUDY")]}),
("I need help in Wirtschaft und Verkehr ", {"entities": [(15,37, "STUDY")]}),
("I did some research in Wirtschaft und Verkehr ", {"entities": [(23,45, "STUDY")]}),
("I want to ask somebody about Wirtschaft und Verkehr ", {"entities": [(29,51, "STUDY")]}),
("I don’t understand Wirtschaft und Verkehr ", {"entities": [(19,41, "STUDY")]}),
("He is an expert in Wirtschaft und Verkehr ", {"entities": [(19,41, "STUDY")]}),
("He is a professor for Wirtschaft und Verkehr ", {"entities": [(22,44, "STUDY")]}),
("I have a test in Wirtschaft und Verkehr ", {"entities": [(17,39, "STUDY")]}),
("He teaches Wirtschaft und Verkehr ", {"entities": [(11,33, "STUDY")]}),
("I need to learn Wirtschaft und Verkehr ", {"entities": [(16,38, "STUDY")]}),
("I have some questions about the topic Wirtschaft und Verkehr ", {"entities": [(38,60, "STUDY")]}),
("I need to do more research about Wirtschaft und Verkehr ", {"entities": [(33,55, "STUDY")]}),
("Which professor is an expert in Wirtschaft und Verkehr ", {"entities": [(32,54, "STUDY")]}),
("Who is an expert in Wirtschaft und Verkehr ", {"entities": [(20,42, "STUDY")]})
]


In [ ]:
# implement function to train ner component to recognize studies
def train_ner_study(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.load("en_core_web_md")
    # Getting the pipeline component
    ner = nlp.get_pipe("ner")
    
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            [text], 
                            [annotations], 
                            drop=0.2,
                            sgd=optimizer,
                            losses=losses
                )
            print(losses)
    return (nlp)
                

In [ ]:
# calling the function
nlp_study = train_ner_study(TRAIN_DATA, 30)

In [ ]:
# save custom model in file
nlp_study.to_disk("custom_ner_model_study")

### 3.10. Funktion für den Chatbot

Hier wird die Funktion erstellt, die alle Kompnenten zur Extraktion weiterer Daten beinhaltet. Je nach identifizierten Intent werden die benötigten Daten extrahiert.

In [75]:
def slot_filling(inp, intent):
    # liste with intents that need information
    intent_telephone = ["prof_name_query_telephone","extract_new_telephone" ]
    intent_email=["prof_name_query_email","extract_new_email" ]
    intent_office=["prof_name_query_office", "extract_new_office"]
    intent_research_area=["prof_name_query_research_area", "extract_new_research_area"]
    intent_study=["prof_name_query_study", "extract_new_study"]
    intent_last_name=["prof_name_query_lastname"]
    intent_first_name=["prof_name_query_firstname"]
    intent_prof_contact=["prof_telephone_query_name", "prof_email_query_name", "prof_office_query_name", "prof_research_area_query_name", "prof_study_query_name"]
    intent_generic_conversation=["greeting","greeting_response","courtesy_greeting","courtesy_greeting_response","real_name_query", "goodbye","task_response"]
    #get the phone number
    if intent in intent_telephone:
        re_number_1 = r"[\d]{2}? [\d]{4} [\d]{3} [\d]{4}"
        re_number_2 = r"[\d]{2} [\(][\d]{1}[\)] [\d]{4} [\d]{3} [\d]{4}"
        extracted_info = re.compile("(%s|%s)" % (re_number_1, re_number_2)).findall(inp)
        
    elif intent in intent_email:
        extracted_info = re.findall('\S+@\S+', inp)
        
    elif intent in intent_office:
        extracted_info = re.findall('[A-Z].\d{1}.\d{2}', inp)
        
    elif intent in intent_research_area:
        doc = nlp_research_area(inp)
        for ent in doc.ents:
            if ent.label_ == "RESEARCH_AREA":
                extracted_info = ent.text    
    
    elif intent in intent_research_area:
        doc = nlp_study(inp)
        for ent in doc.ents:
            if ent.label_ == "STUDY":
                extracted_info = ent.text  
                
    elif intent in intent_last_name:
        for last_name in last_names:
            for word in inp.split():
                if SequenceMatcher(None, last_name, word).ratio() >= 0.7:
                    extracted_info = last_name
    
    elif intent in intent_first_name:
        for first_name in first_names:
            for word in inp.split():
                if SequenceMatcher(None, first_name, word).ratio() >= 0.7:
                    extracted_info = first_name
        
    elif intent in intent_prof_contact:
        for last_name in last_names:
            for word in inp.split():
                if SequenceMatcher(None, last_name, word).ratio() >= 0.7:
                    extracted_info = last_name
                    
    elif intent in intent_generic_response:
        extracted_info = "generic intent no need for info extraction"
    
    return extracted_info

### 3.10. Herausfoderungen 

* Manuelles Erstellen der Trainingdaten notwendig
* Probleme beim Traineren der spacy Modelle. Selbsttrainerte Modelle erkennen nur noch die neu erlernte Entität. Sonst keine mehr
* Benutzer muss Informationen exakt, wie in den Regular Expression Patterns definiert, schreiben, sonst können Telefonnummern usw. nicht erkannt werden --> nicht robust gegen Rechtschreibfehler